In [214]:
import pandas as pd
import numpy as np

# to desable warnings
pd.options.mode.chained_assignment = None  # default='warn'

from math import fabs
# Отличие abs от fabs заключается в том, что функция abs возвращает значение того же типа, что и аргумент. 
# Функция же fabs вначале преобразует тип аргумента к вещественному числу.

from datetime import datetime

from bs4 import BeautifulSoup
import requests

from decimal import Decimal

In [323]:
# D:\!HRYTS_FOLDER\2023-REPORT
# D:/!HRYTS_FOLDER/2023-REPORT/2023_export_input.xlsx
# D:/!HRYTS_FOLDER/2023-REPORT/2023_dostavka.xlsx
# D:/!HRYTS_FOLDER/2023-REPORT/2023_contracts.xlsx
df_otgruz = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/2023_export_input.xlsx', 
                          usecols='A,B,C,G,H,I,J,K,L,M,N,O, S'
                         )
df_otgruz["Дата выпуска"] = df_otgruz["Дата выпуска"].dt.strftime('%d-%m-%Y')
df_otgruz["Дата отгрузки"] = df_otgruz["Дата отгрузки"].dt.strftime('%d-%m-%Y')
df_otgruz["Дата контракта/инвойса/спецификации"] = df_otgruz["Дата контракта/инвойса/спецификации"].dt.strftime('%d-%m-%Y')
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик
0,13-01-2023,23-001,12-01-2023,4,РСА-10-350V-PD rev.SHA 1.3 (№8090-8133),1571.0,25-10-2022,44,25912.82,455.00,SHARPLIGHT TECHNOLOGIES CO LTD,Израиль,able
1,13-01-2023,23-001,12-01-2023,11,РСА-10-500V-PD rev.VIORA 2.7 (№8357-8378),1616.0,27-12-2022,11,7094.53,455.00,SHARPLIGHT TECHNOLOGIES CO LTD,Израиль,able
2,13-01-2023,23-001,12-01-2023,11,РСА-10-500V-PD rev.VIORA 2.7 (№8357-8378),1618.0,11-01-2023,11,7036.44,455.00,SHARPLIGHT TECHNOLOGIES CO LTD,Израиль,able
3,17-01-2023,23-002,16-01-2023,10,QBU-4010-ALMA rev.STD 1.0 (№1101-1110),1634.0,16-01-2023,10,2702.07,400.00,ALMA Lasers,Израиль,able
4,24-01-2023,(22-186),10-10-2022,20,"РСА-40-1000V-PD rev.STD 3.1 Jofre (№0750-0768,...",1635.0,23-01-2023,20,35131.19,1101.42,ООО «Медицинские Оптические Технологии»,РФ,dpd


In [213]:
# pd.to_datetime(df_otgruz["Дата контракта/инвойса/спецификации"], errors='coerce').tail(20)
# ---------------------------------------------------------------------------------------------

In [216]:
# инвойс - перевозчик за 2 года
df_otgruz_transp = df_otgruz[
    ['№ инвойса (для РБ, РФ в скобках)', 'Дата отгрузки',
     'Перевозчик']
].drop_duplicates()
df_otgruz_transp.head()

,"№ инвойса (для РБ, РФ в скобках)",Дата отгрузки,Перевозчик
0,23-001,13-01-2023,able
3,23-002,17-01-2023,able
4,(22-186),24-01-2023,dpd
5,(22-188-1),30-01-2023,самовывоз
6,23-004,01-02-2023,able


In [17]:
df_otgruz_transp.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_otgruz_transp.xlsx', index=False) 

In [ ]:
# ------------------------------------------------------------------------

In [324]:
df_otgruz.tail(10)

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик
304,05-11-2024,24-085,01-11-2024,1,HVPS-300-2kV rev.STD 1.4 (0055),1960.0,18-10-2024,1,724.90,500.0,"Coupletech Co., Ltd.",Китай,able
305,05-11-2024,24-085,01-11-2024,2,QBY-4010 Разработка 08сч. - QBY-4010 rev.STD 1...,1953.0,09-10-2024,2,1561.38,1500.0,"Coupletech Co., Ltd.",Китай,able
306,05-11-2024,24-085,01-11-2024,1,RFGM-6.78-800 STD 1.2.1 (0012),1968.0,31-10-2024,1,2394.29,1050.0,"Coupletech Co., Ltd.",Китай,able
307,05-11-2024,24-008-2,03-09-2024,30,Медицинский зарядный модуль РСА-20-1200V-CD re...,1929.0,25-07-2024,49,54772.13,700.0,"Accu-Tech Co., Ltd.",Китай,able
308,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able
309,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd
310,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd
311,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd
312,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз
313,15-11-2024,24-090,14-11-2024,3,NaN,1974.0,14-11-2024,3,NaN,700.0,"L2K (Laser Leader of Korea) Co., Ltd",Корея,able


In [325]:
# отсечь несделанные отгрузки. или без выпуска, или за прошлые годы
df_otgruz.iloc[150:-1].tail()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик
308,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able
309,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd
310,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd
311,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd
312,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз


In [326]:
# отсечь несделанные отгрузки. или без выпуска, или за прошлые годы
df_otgruz = df_otgruz.iloc[150:-1]

In [327]:
df_otgruz[df_otgruz['№ инвойса (для РБ, РФ в скобках)'] == '(24-004)']

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик
165,01-02-2024,(24-004),12-01-2024,1,QBD-mini-1004-UP rev.STD 1.1 (№0300),1836.0,26-01-2024,1,99.75,600.0,"ООО ""Кристаллы Сибири""",РФ,dpd


In [328]:
df_contracts = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/2023_contracts.xlsx')
df_contracts["Дата прихода на счет"] = df_contracts["Дата прихода на счет"].dt.strftime('%d-%m-%Y')
df_contracts.head()

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Unnamed: 5
0,23-001,usd,12480.0,28-02-2023,12480.00,NaN
1,23-002,usd,4400.0,03-04-2023,4400.00,NaN
2,(22-186),rub,NaN,NaN,1354000.00,NaN
3,(22-188-1),byn,NaN,NaN,20213.35,NaN
4,23-004,usd,2300.0,24-03-2023,2300.00,NaN


In [329]:
# если контракт в долларах или оплачен, его можно вносить и не обрезать !!!

# df_contracts = df_contracts.iloc[0:-1]
df_contracts.tail(3)

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Unnamed: 5
220,(24-087),rub,292404.00,13-11-2024,NaN,NaN
221,(24-088),byn,13962.76,13-11-2024,NaN,NaN
222,24-090,eur,NaN,NaN,2307.0,NET30


In [330]:
df_dostavka = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/2023_dostavka.xlsx')
df_dostavka["дата выставления счета"] = df_dostavka["дата выставления счета"].dt.strftime('%d-%m-%Y')
df_dostavka.head(30)

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)",Unnamed: 4
0,NaN,NaN,able,24-090,NaN
1,NaN,NaN,able,24-090,NaN
2,NaN,50.000,prior,24-090,NaN
3,NaN,NaN,DT,24-090,NaN
4,NaN,NaN,SDT,(24-086),NaN
5,NaN,NaN,dpd,(24-086),NaN
6,15-11-2024,2653.280,able,24-008-2,NaN
7,15-11-2024,107.340,able,24-008-2,NaN
8,06-11-2024,75.000,DT,24-008-2,NaN
9,15-11-2024,1308.950,able,24-085,NaN


In [331]:
# -2 к номеру строки в excel файле
df_dostavka.iloc[6:].head()

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)",Unnamed: 4
6,15-11-2024,2653.28,able,24-008-2,NaN
7,15-11-2024,107.34,able,24-008-2,NaN
8,06-11-2024,75.00,DT,24-008-2,NaN
9,15-11-2024,1308.95,able,24-085,NaN
10,15-11-2024,107.34,able,24-085,NaN


In [332]:
# отбрасываем верхние строки, где неполные оплаты по доставке
df_dostavka = df_dostavka.iloc[6:]
df_dostavka.head(3)

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)",Unnamed: 4
6,15-11-2024,2653.28,able,24-008-2,NaN
7,15-11-2024,107.34,able,24-008-2,NaN
8,06-11-2024,75.00,DT,24-008-2,NaN


In [333]:
df_contract_date = df_otgruz[['№ инвойса (для РБ, РФ в скобках)', 'Дата контракта/инвойса/спецификации']].drop_duplicates()
df_contract_date.head()

,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации
150,23-031-dc,14-12-2023
151,(23-090),29-11-2023
152,24-001-1,11-01-2024
153,24-001-2,11-01-2024
154,23-008-4,08-01-2024


In [334]:
# filtered_modul_df = report_dict_df.merge(self.df_from_file, on='Артикул', how='left')
df_contracts_1 = df_contracts.merge(df_contract_date, on='№ инвойса (для РБ, РФ в скобках)', how='left')
print(df_contracts_1.shape)
print(df_contracts.shape)
df_contracts_1.head()

(223, 7)
(223, 6)


,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Unnamed: 5,Дата контракта/инвойса/спецификации
0,23-001,usd,12480.0,28-02-2023,12480.00,NaN,NaN
1,23-002,usd,4400.0,03-04-2023,4400.00,NaN,NaN
2,(22-186),rub,NaN,NaN,1354000.00,NaN,NaN
3,(22-188-1),byn,NaN,NaN,20213.35,NaN,NaN
4,23-004,usd,2300.0,24-03-2023,2300.00,NaN,NaN


In [335]:
df_contracts_1.tail()

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Unnamed: 5,Дата контракта/инвойса/спецификации
218,24-085,cny,NaN,NaN,153225.0,NET30,01-11-2024
219,(24-086),rub,1632162.00,06-11-2024,1632162.0,NaN,04-11-2024
220,(24-087),rub,292404.00,13-11-2024,NaN,NaN,NaN
221,(24-088),byn,13962.76,13-11-2024,NaN,NaN,NaN
222,24-090,eur,NaN,NaN,2307.0,NET30,NaN


In [17]:
# если внесены оплаты по неотгруженным контрактам - нужно их отбросить ???
# df_contracts_1[:-2].tail()

In [18]:
# df_contracts_1 = df_contracts_1[:-2]  # ????

In [336]:
df_date_of_contract = df_otgruz[['Дата контракта/инвойса/спецификации']].drop_duplicates()
df_date_of_contract = df_date_of_contract.rename(columns={"Дата контракта/инвойса/спецификации": "date"})

df_date_of_payment = df_contracts[['Дата прихода на счет']].drop_duplicates()
df_date_of_payment = df_date_of_payment.rename(columns={"Дата прихода на счет": "date"})

df_date_dostavka = df_dostavka[['дата выставления счета']].drop_duplicates()
df_date_dostavka = df_date_dostavka.rename(columns={"дата выставления счета": "date"})

df_date_dostavka.head()

,date
6,15-11-2024
8,06-11-2024
13,05-11-2024
15,31-10-2024
21,30-10-2024


In [337]:
# 'Дата отгрузки', 'Дата контракта/инвойса/спецификации', 'Дата выпуска'
df_date_otgruz = df_otgruz[['Дата отгрузки']].drop_duplicates()
df_date_otgruz = df_date_otgruz.rename(columns={"Дата отгрузки": "date"})

df_date_invoice = df_otgruz[['Дата контракта/инвойса/спецификации']].drop_duplicates()
df_date_invoice = df_date_invoice.rename(columns={"Дата контракта/инвойса/спецификации": "date"})

df_date_vypusk = df_otgruz[['Дата выпуска']].drop_duplicates()
df_date_vypusk = df_date_vypusk.rename(columns={"Дата выпуска": "date"})

df_dates = pd.concat([df_date_of_contract, df_date_of_payment, df_date_dostavka, df_date_otgruz, df_date_invoice, df_date_vypusk])
print(df_dates.shape)
df_dates = df_dates.drop_duplicates()
df_dates = df_dates.dropna()
print(df_dates.shape)
df_exchange_rate_by_date = df_dates
print(df_exchange_rate_by_date.shape)

df_dates.head()

(618, 1)
(329, 1)
(329, 1)


,date
150,14-12-2023
151,29-11-2023
152,11-01-2024
154,08-01-2024
158,22-01-2024


In [338]:
df_dates.tail()

,date
265,12-08-2024
284,26-09-2024
288,14-10-2024
291,18-10-2024
312,11-11-2024


In [22]:
# make df_dates_old FIRST !!!!

In [23]:
#df_dates.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_dates.xlsx')

In [339]:
# date update     ???????
df_dates_old = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_dates.xlsx')
df_dates_old.head(3)

,Unnamed: 0,date
0,150,14-12-2023
1,151,29-11-2023
2,152,11-01-2024


In [340]:
df_dates_old.tail(3)

,Unnamed: 0,date
326,288,14-10-2024
327,291,18-10-2024
328,312,11-11-2024


In [341]:
df_dates_old = df_dates_old[['date']]
df_dates_old.head(3)

,date
0,14-12-2023
1,29-11-2023
2,11-01-2024


In [342]:
print(df_dates.shape)
print(df_dates_old.shape)

(329, 1)
(329, 1)


In [343]:
# find diff between dates - drop duplicates after concat
df_dates_concat = pd.concat([df_dates, df_dates_old])
print(df_dates_concat.shape)

(658, 1)


In [344]:
df_dates_diff = df_dates_concat.drop_duplicates(subset=['date'], keep=False)
print(df_dates_diff.shape)
df_dates_diff

(0, 1)


,date


In [238]:
df_dates.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_dates.xlsx')

In [239]:
def get_currency_data(date):
        
    url = f"https://myfin.by/bank/kursy_valjut_nbrb/{date}?"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')  
    
    arr = [i for i in soup.find_all("div", {"class": "table-responsive"})]
    
    arr_currency = [[j.text.strip() for j in i.find_all("td")] for i in arr]
    list_currency = []
    [[list_currency.append(j) for j in arr_currency[i]] for i in range(0, len(arr_currency))]
    
    list_c = [list_currency[i:i+4] for i in range(0,28,4)]
    
    return list_c

In [240]:
# ДОЛГИЙ СКРИПТ! на несколько минут !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# для запуска в 1й раз!

df_exchange_rate_by_date['list_of_exchanges'] = df_exchange_rate_by_date.apply(
    lambda x: get_currency_data(x['date']), axis=1) 

df_exchange_rate_by_date.head(3)

,date,list_of_exchanges
150,14-12-2023,"[[Доллар США, 3.166, USD, 1], [Евро, 3.4146, E..."
151,29-11-2023,"[[Доллар США, 3.103, USD, 1], [Евро, 3.3987, E..."
152,11-01-2024,"[[Доллар США, 3.176, USD, 1], [Евро, 3.4765, E..."


In [241]:
df_exchange_rate_by_date.tail(3)

,date,list_of_exchanges
155,28-07-2023,"[[Доллар США, 3.0057, USD, 1], [Евро, 3.3494, ..."
156,11-09-2023,"[[Доллар США, 3.2777, USD, 1], [Евро, 3.5085, ..."
175,05-02-2024,"[[Доллар США, 3.2113, USD, 1], [Евро, 3.497, E..."


In [240]:
# for update
df_dates_diff = df_dates_diff.dropna()
df_exchange_rate_by_date_diff = df_dates_diff[['date']]

df_exchange_rate_by_date_diff['list_of_exchanges'] = df_exchange_rate_by_date_diff.apply(
    lambda x: get_currency_data(x['date']), axis=1) 

df_exchange_rate_by_date_diff.head()

,date,list_of_exchanges
171,21-02-2169,"[[Доллар США, 3.3635, 3.3751 +0.0116, USD], [1..."


In [241]:
df_exchange_rate_by_date_diff.tail()

,date,list_of_exchanges
171,21-02-2169,"[[Доллар США, 3.3635, 3.3751 +0.0116, USD], [1..."


In [243]:
# save data 1st time
df_exchange_rate_by_date.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_exchange_rate_by_date.xlsx')  

In [345]:
# load data
df_exchange_rate_by_date = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_exchange_rate_by_date.xlsx')
df_exchange_rate_by_date = df_exchange_rate_by_date.drop(columns=['Unnamed: 0'])

In [346]:
df_exchange_rate_by_date.head(3)

,date,list_of_exchanges
0,14-12-2023,"[['Доллар США', '3.166', 'USD', '1'], ['Евро',..."
1,29-11-2023,"[['Доллар США', '3.103', 'USD', '1'], ['Евро',..."
2,11-01-2024,"[['Доллар США', '3.176', 'USD', '1'], ['Евро',..."


In [347]:
df_exchange_rate_by_date[df_exchange_rate_by_date['date'] == '06-05-2024']

,date,list_of_exchanges
212,06-05-2024,"[['Доллар США', '3.2267', 'USD', '1'], ['Евро'..."


In [348]:
df_exchange_rate_by_date['list_of_exchanges']

0      [['Доллар США', '3.166', 'USD', '1'], ['Евро',...
1      [['Доллар США', '3.103', 'USD', '1'], ['Евро',...
2      [['Доллар США', '3.176', 'USD', '1'], ['Евро',...
3      [['Доллар США', '3.1986', 'USD', '1'], ['Евро'...
4      [['Доллар США', '3.1637', 'USD', '1'], ['Евро'...
                             ...                        
324    [['Доллар США', '3.3407', 'USD', '1'], ['Евро'...
325    [['Доллар США', '3.2133', 'USD', '1'], ['Евро'...
326    [['Доллар США', '3.2832', 'USD', '1'], ['Евро'...
327    [['Доллар США', '3.3119', 'USD', '1'], ['Евро'...
328    [['Доллар США', '3.3312', 'USD', '1'], ['Евро'...
Name: list_of_exchanges, Length: 329, dtype: object

In [349]:
# convert to list the str-type column of lists
df_exchange_rate_by_date['list_of_exchanges'] = df_exchange_rate_by_date.apply(
    lambda x: eval(x['list_of_exchanges']), axis=1) 

# list_c_test = df_exchange_rate_by_date['list_of_exchanges'][df_exchange_rate_by_date['date'] == '22-03-2023']
# type(list_c_test.iloc[0])

In [350]:
df_exchange_rate_by_date['list_of_exchanges']

0      [[Доллар США, 3.166, USD, 1], [Евро, 3.4146, E...
1      [[Доллар США, 3.103, USD, 1], [Евро, 3.3987, E...
2      [[Доллар США, 3.176, USD, 1], [Евро, 3.4765, E...
3      [[Доллар США, 3.1986, USD, 1], [Евро, 3.4906, ...
4      [[Доллар США, 3.1637, USD, 1], [Евро, 3.4429, ...
                             ...                        
324    [[Доллар США, 3.3407, USD, 1], [Евро, 3.5678, ...
325    [[Доллар США, 3.2133, USD, 1], [Евро, 3.5949, ...
326    [[Доллар США, 3.2832, USD, 1], [Евро, 3.5909, ...
327    [[Доллар США, 3.3119, USD, 1], [Евро, 3.596, E...
328    [[Доллар США, 3.3312, USD, 1], [Евро, 3.5806, ...
Name: list_of_exchanges, Length: 329, dtype: object

In [351]:
df_exchange_rate_by_date_new = pd.concat([df_exchange_rate_by_date, df_exchange_rate_by_date_diff])
print(df_exchange_rate_by_date_new.shape)
df_exchange_rate_by_date_new = df_exchange_rate_by_date_new.drop_duplicates(subset=['date'], keep=False)
print(df_exchange_rate_by_date_new.shape)

(330, 2)
(330, 2)


In [249]:
# save new data
df_exchange_rate_by_date = df_exchange_rate_by_date_new
df_exchange_rate_by_date.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_exchange_rate_by_date.xlsx')  

In [250]:
df_exchange_rate_by_date = df_exchange_rate_by_date_new

In [352]:
df_exchange_rate_by_date

,date,list_of_exchanges
0,14-12-2023,"[[Доллар США, 3.166, USD, 1], [Евро, 3.4146, E..."
1,29-11-2023,"[[Доллар США, 3.103, USD, 1], [Евро, 3.3987, E..."
2,11-01-2024,"[[Доллар США, 3.176, USD, 1], [Евро, 3.4765, E..."
3,08-01-2024,"[[Доллар США, 3.1986, USD, 1], [Евро, 3.4906, ..."
4,22-01-2024,"[[Доллар США, 3.1637, USD, 1], [Евро, 3.4429, ..."
...,...,...
324,12-11-2024,"[[Доллар США, 3.3407, USD, 1], [Евро, 3.5678, ..."
325,26-09-2024,"[[Доллар США, 3.2133, USD, 1], [Евро, 3.5949, ..."
326,14-10-2024,"[[Доллар США, 3.2832, USD, 1], [Евро, 3.5909, ..."
327,18-10-2024,"[[Доллар США, 3.3119, USD, 1], [Евро, 3.596, E..."


In [353]:
df_exchange_rate_by_date[df_exchange_rate_by_date['date'] == '02-05-2024']

,date,list_of_exchanges
214,02-05-2024,"[[Доллар США, 3.255, USD, 1], [Евро, 3.4916, E..."


In [7]:
# ОК, но не надо

def to_usd(date, price, currency='byn'):
    
#     num = Decimal('2.675')
    if currency == 'usd':
        return float(price)
    
    url = f"https://myfin.by/bank/kursy_valjut_nbrb/{date}?"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')  
    
    arr = [i for i in soup.find_all("div", {"class": "table-responsive"})]
    arr_currency = [[j.text.strip() for j in i.find_all("td")] for i in arr]
    list_currency = []
    [[list_currency.append(j) for j in arr_currency[i]] for i in range(0, len(arr_currency))]
    
    list_c = [list_currency[i:i+4] for i in range(0,12,4)]
    
    
    if currency == 'byn':
        return float(price)/float(list_c[0][1])
    
    if currency == 'eur':
        return float(price)*float(list_c[1][1])/float(list_c[0][1])
    
    if currency == 'rub':
        return float(price)*float(list_c[2][1])/(100*float(list_c[0][1]))
    
    return None

In [354]:
# НАДО

def to_usd_by_date(date, price, currency='byn'):    
    
#     num = Decimal('2.675')
    if currency == 'usd':
        return float(price)
    
#     print(date)
    try:
        list_c = df_exchange_rate_by_date['list_of_exchanges'][df_exchange_rate_by_date['date'] == date].iloc[0]
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}, date = {date}")
        print(df_exchange_rate_by_date['list_of_exchanges'][df_exchange_rate_by_date['date'] == date])
#     print(list_c)
#     print(type(list_c))
    
    if currency == 'byn':
        return float(price)/float(list_c[0][1])
    
    if currency == 'eur':
        return float(price)*float(list_c[1][1])/float(list_c[0][1])
    
    if currency == 'rub':
        return float(price)*float(list_c[2][1])/(100*float(list_c[0][1]))
    
    if currency == 'cny':
        return float(price)*float(list_c[6][1])/(10*float(list_c[0][1]))
    
    return None

In [288]:
# CALCULATOR валюта ==> $ !!!!!!!!!!!!!!!!!!!!!!!!!!! price = 1 --> exchange rate
# to_usd_by_date('16-05-2024', 1, 'rub')
91.66 * to_usd_by_date('08-05-2024', 1, 'rub')

0.999916928901716

In [287]:
# CALCULATOR 1$ = курс валюты НБ РБ на дату !!!!!!!!!!!!!!!!!!!!!!!!!!! price = 1$ --> exchange rate
1/to_usd_by_date('08-05-2024', 1, 'rub')

91.66761492944926

In [162]:
# # using decimal.Decimal (passed float as string for precision)
# num = Decimal('2.675')
# print(round(num, 2))

2.68


In [355]:
df_contracts_1.head()

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Unnamed: 5,Дата контракта/инвойса/спецификации
0,23-001,usd,12480.0,28-02-2023,12480.00,NaN,NaN
1,23-002,usd,4400.0,03-04-2023,4400.00,NaN,NaN
2,(22-186),rub,NaN,NaN,1354000.00,NaN,NaN
3,(22-188-1),byn,NaN,NaN,20213.35,NaN,NaN
4,23-004,usd,2300.0,24-03-2023,2300.00,NaN,NaN


In [356]:
df_contracts_1[['Факт.сумма, $']] = np.nan

In [357]:
df_contracts_1.head(10)

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Unnamed: 5,Дата контракта/инвойса/спецификации,"Факт.сумма, $"
0,23-001,usd,12480.00,28-02-2023,12480.00,NaN,NaN,NaN
1,23-002,usd,4400.00,03-04-2023,4400.00,NaN,NaN,NaN
2,(22-186),rub,NaN,NaN,1354000.00,NaN,NaN,NaN
3,(22-188-1),byn,NaN,NaN,20213.35,NaN,NaN,NaN
4,23-004,usd,2300.00,24-03-2023,2300.00,NaN,NaN,NaN
5,(22-188-2),byn,21768.45,23-01-2023,21768.45,NaN,NaN,NaN
6,22-215,usd,31000.00,23-12-2022,NaN,NaN,NaN,NaN
7,23-007,usd,NaN,NaN,2270.00,paid (31700 17.04.2023),NaN,NaN
8,23-011,usd,10600.00,24-03-2023,10600.00,NaN,NaN,NaN
9,23-015,usd,6040.00,30-06-2023,6040.00,NaN,NaN,NaN


In [358]:
df_contracts_1 = df_contracts_1.drop(columns=['Unnamed: 5'])
#df_contracts_1 = df_contracts_1.drop(columns=['Unnamed: 6'])

In [359]:
df_contracts_1.head(3)

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Дата контракта/инвойса/спецификации,"Факт.сумма, $"
0,23-001,usd,12480.0,28-02-2023,12480.0,NaN,NaN
1,23-002,usd,4400.0,03-04-2023,4400.0,NaN,NaN
2,(22-186),rub,NaN,NaN,1354000.0,NaN,NaN


In [360]:
# df_contracts_1.shape

In [361]:
# df_contracts_1['Факт.сумма, $'][
#     (df_contracts_1['Сумма'].notnull())]

In [362]:
# df_contracts_1['Факт.сумма, $'][
#     (df_contracts_1['Сумма'].notnull())].shape

In [363]:
# df_contracts_1[
#     (df_contracts_1['Сумма'].notnull())]

In [364]:
# важная проверка (должно быть пустым)
df_contracts_1[(df_contracts_1['Дата прихода на счет'].isnull())&(df_contracts_1['Сумма'].notnull())]

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Дата контракта/инвойса/спецификации,"Факт.сумма, $"


In [365]:
# df_contracts_1[
#     (df_contracts_1['Сумма'].notnull())].shape

In [366]:
# если деньги на счет пришли ( ['Сумма'].notnull() ), берутся они, если нет ( ['Сумма'].isnull() ), 
# то сумма, указанная в инвойсе

# df_contracts_1['Факт.сумма, $'][
#     (df_contracts_1['Сумма'].notnull())] = df_contracts_1[
#     (df_contracts_1['Сумма'].notnull())].apply(
#     lambda x: to_usd(x['Дата прихода на счет'], x['Сумма'], x['Валюта']), axis=1)

df_contracts_1['Факт.сумма, $'][
    (df_contracts_1['Сумма'].notnull())] = df_contracts_1[
    (df_contracts_1['Сумма'].notnull())].apply(
    lambda x: to_usd_by_date(x['Дата прихода на счет'], x['Сумма'], x['Валюта']), axis=1)

In [367]:
# убираем неактуальные контракты
df_contracts_1 = df_contracts_1.dropna(subset=['Дата контракта/инвойса/спецификации'])

In [368]:
# df_contracts_1['Факт.сумма, $'][
#     (df_contracts_1['Сумма'].isnull())] = df_contracts_1[
#     (df_contracts_1['Сумма'].isnull())].apply(
#     lambda x: to_usd(x['Дата контракта/инвойса/спецификации'], x['Сумма контр-та'], x['Валюта']), axis=1)

df_contracts_1['Факт.сумма, $'][
    (df_contracts_1['Сумма'].isnull())] = df_contracts_1[
    (df_contracts_1['Сумма'].isnull())].apply(
    lambda x: to_usd_by_date(x['Дата контракта/инвойса/спецификации'], x['Сумма контр-та'], x['Валюта']), axis=1)

In [369]:
df_contracts_1

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Дата контракта/инвойса/спецификации,"Факт.сумма, $"
55,23-008-4,usd,5455.800,29-03-2023,NaN,08-01-2024,5455.800000
60,23-008-4,cny,219672.560,18-01-2024,30940.0,08-01-2024,30292.603100
72,23-010-3,usd,5270.400,10-03-2023,NaN,06-03-2024,5270.400000
77,23-010-3,cny,212093.540,22-03-2024,29865.6,06-03-2024,29362.612990
89,23-066-5,cny,33245.465,12-10-2023,4635.0,22-01-2024,4561.986267
...,...,...,...,...,...,...,...
215,24-083,cny,NaN,NaN,31208.5,25-10-2024,4405.266020
216,24-084,eur,NaN,NaN,9276.0,28-10-2024,10039.409313
217,23-016-dc,usd,NaN,NaN,0.0,05-11-2024,0.000000
218,24-085,cny,NaN,NaN,153225.0,01-11-2024,21587.897141


In [370]:
df_contracts_1.tail(10)

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Дата контракта/инвойса/спецификации,"Факт.сумма, $"
209,(24-075),rub,109596.00,02-10-2024,NaN,25-09-2024,1184.437800
210,24-076,eur,NaN,NaN,3732.0,07-10-2024,4115.595991
211,(22-188-3),byn,5780.34,09-10-2024,NaN,30-09-2024,1759.509315
213,24-079,cny,NaN,NaN,10071.0,15-10-2024,1430.059346
214,(22-188-4),byn,38201.12,23-10-2024,NaN,23-10-2024,11600.000000
215,24-083,cny,NaN,NaN,31208.5,25-10-2024,4405.266020
216,24-084,eur,NaN,NaN,9276.0,28-10-2024,10039.409313
217,23-016-dc,usd,NaN,NaN,0.0,05-11-2024,0.000000
218,24-085,cny,NaN,NaN,153225.0,01-11-2024,21587.897141
219,(24-086),rub,1632162.00,06-11-2024,1632162.0,04-11-2024,16727.162593


In [371]:
df_contracts_1[df_contracts_1['№ инвойса (для РБ, РФ в скобках)'] == '(24-073)']

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Дата контракта/инвойса/спецификации,"Факт.сумма, $"
206,(24-073),rub,98304.0,23-09-2024,NaN,18-09-2024,1061.800607


In [372]:
df_contracts_1[df_contracts_1['№ инвойса (для РБ, РФ в скобках)'] == '24-083']

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Дата прихода на счет,Сумма контр-та,Дата контракта/инвойса/спецификации,"Факт.сумма, $"
215,24-083,cny,NaN,NaN,31208.5,25-10-2024,4405.26602


In [373]:
df_contracts_2 = df_contracts_1[['№ инвойса (для РБ, РФ в скобках)','Валюта', 'Факт.сумма, $']].groupby(
    by=['№ инвойса (для РБ, РФ в скобках)','Валюта'], as_index=False).sum()

In [374]:
df_contracts_2

,"№ инвойса (для РБ, РФ в скобках)",Валюта,"Факт.сумма, $"
0,(22-188-3),byn,1759.509315
1,(22-188-4),byn,11600.000000
2,(23-087),rub,25896.961439
3,(23-090),byn,448.990354
4,(24-004),rub,650.782320
...,...,...,...
82,24-076,eur,4115.595991
83,24-079,cny,1430.059346
84,24-083,cny,4405.266020
85,24-084,eur,10039.409313


In [375]:
print(df_contracts_1.shape, df_contracts_2.shape)

(96, 7) (87, 3)


In [376]:
# РАСХОДЫ НА ТРАНСПОРТ
df_dostavka.head()

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)",Unnamed: 4
6,15-11-2024,2653.28,able,24-008-2,NaN
7,15-11-2024,107.34,able,24-008-2,NaN
8,06-11-2024,75.00,DT,24-008-2,NaN
9,15-11-2024,1308.95,able,24-085,NaN
10,15-11-2024,107.34,able,24-085,NaN


In [377]:
# убрать неактуальный транспорт - ЗА КОНТРАКТЫ ПРОШЛОГО ГОДА
df_dostavka[:-202].tail()

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)",Unnamed: 4
223,22-01-2024,5.64,BELPOCHTA,24-001-2,NaN
224,22-01-2024,221.34,EMS,24-001-2,NaN
225,11-01-2024,66.30,DT,23-031-dc,NaN
226,11-01-2024,5.64,BELPOCHTA,23-031-dc,NaN
227,11-01-2024,130.86,EMS,23-031-dc,NaN


In [378]:
# убрать неактуальный транспорт - ЗА КОНТРАКТЫ ПРОШЛОГО ГОДА
df_dostavka = df_dostavka[:-202]

In [379]:
df_dostavka.tail()

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)",Unnamed: 4
223,22-01-2024,5.64,BELPOCHTA,24-001-2,NaN
224,22-01-2024,221.34,EMS,24-001-2,NaN
225,11-01-2024,66.30,DT,23-031-dc,NaN
226,11-01-2024,5.64,BELPOCHTA,23-031-dc,NaN
227,11-01-2024,130.86,EMS,23-031-dc,NaN


In [380]:
#df_contracts_1 = df_contracts_1.drop(columns=['Unnamed: 5'])
df_dostavka = df_dostavka.drop(columns=['Unnamed: 4'])
df_dostavka

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)"
6,15-11-2024,2653.28,able,24-008-2
7,15-11-2024,107.34,able,24-008-2
8,06-11-2024,75.00,DT,24-008-2
9,15-11-2024,1308.95,able,24-085
10,15-11-2024,107.34,able,24-085
...,...,...,...,...
223,22-01-2024,5.64,BELPOCHTA,24-001-2
224,22-01-2024,221.34,EMS,24-001-2
225,11-01-2024,66.30,DT,23-031-dc
226,11-01-2024,5.64,BELPOCHTA,23-031-dc


In [381]:
# df_dostavka['сумма, $'] = df_dostavka.apply(
#     lambda x: to_usd(x['дата выставления счета'], x['сумма, byn']), axis=1)

df_dostavka['сумма, $'] = df_dostavka.apply(
    lambda x: to_usd_by_date(x['дата выставления счета'], x['сумма, byn']), axis=1)

In [382]:
df_dostavka.head()

,дата выставления счета,"сумма, byn",за что оплата,"№ инвойса (для РБ, РФ в скобках)","сумма, $"
6,15-11-2024,2653.28,able,24-008-2,788.844953
7,15-11-2024,107.34,able,24-008-2,31.913186
8,06-11-2024,75.00,DT,24-008-2,22.506302
9,15-11-2024,1308.95,able,24-085,389.163074
10,15-11-2024,107.34,able,24-085,31.913186


In [383]:
df_dostavka_by_contract = df_dostavka[
    ['№ инвойса (для РБ, РФ в скобках)', 'сумма, $']
].groupby('№ инвойса (для РБ, РФ в скобках)', as_index=False).sum()

print(df_dostavka_by_contract.shape)
df_dostavka_by_contract.head()

(74, 2)


,"№ инвойса (для РБ, РФ в скобках)","сумма, $"
0,(23-071-rem),13.792670
1,(23-087),165.902782
2,(24-004),14.270570
3,(24-010),20.947306
4,(24-011),141.184503


In [384]:
df_dostavka_by_contract

,"№ инвойса (для РБ, РФ в скобках)","сумма, $"
0,(23-071-rem),13.792670
1,(23-087),165.902782
2,(24-004),14.270570
3,(24-010),20.947306
4,(24-011),141.184503
...,...,...
69,24-076,278.322337
70,24-079,460.247100
71,24-083,333.349757
72,24-084,262.202548


In [ ]:
# бух стоимость $ за шт и коэф-т

In [385]:
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик
150,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems
151,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз
152,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems
153,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,"Coupletech Co., Ltd.",Китай,ems
154,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,"Accu-Tech Co., Ltd.",Китай,able


In [386]:
# df_otgruz['бух, $'] = df_otgruz.apply(
#     lambda x: to_usd(x['Дата выпуска'], x['общая цена выпуска, BYN']), axis=1)

df_otgruz['бух, $'] = df_otgruz.apply(
    lambda x: to_usd_by_date(x['Дата выпуска'], x['общая цена выпуска, BYN']), axis=1)

df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $"
150,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242
151,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662
152,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233
153,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,"Coupletech Co., Ltd.",Китай,ems,1123.513546
154,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,"Accu-Tech Co., Ltd.",Китай,able,21140.895052


In [387]:
df_otgruz.tail()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $"
308,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able,7283.079358
309,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd,4696.795535
310,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd,7749.647258
311,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd,2785.623414
312,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз,7416.069284


In [388]:
# # using decimal.Decimal (passed float as string for precision)
# num = Decimal('2.675')
# print(round(num, 2))

df_otgruz['бух, $/шт'] = df_otgruz['бух, $'] / df_otgruz['кол-во в выпуске, шт.']
df_otgruz['бух, $/шт'] = df_otgruz['бух, $/шт'].apply(lambda x: round(Decimal(f'{x}'), 2))
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт"
150,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242,215.39
151,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662,57.54
152,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233,1260.13
153,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,"Coupletech Co., Ltd.",Китай,ems,1123.513546,224.70
154,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,"Accu-Tech Co., Ltd.",Китай,able,21140.895052,422.82


In [389]:
df_otgruz[['Произв. расходы - множитель']] = np.nan

In [390]:
df_otgruz['Произв. расходы - множитель'][df_otgruz['кол-во в выпуске, шт.']<3] = 1
df_otgruz['Произв. расходы - множитель'][(df_otgruz['кол-во в выпуске, шт.']>2)&(df_otgruz['кол-во в выпуске, шт.']<8)] = 0.75
df_otgruz['Произв. расходы - множитель'][df_otgruz['кол-во в выпуске, шт.']>7] = 0.5
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель
150,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242,215.39,1.00
151,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662,57.54,1.00
152,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233,1260.13,1.00
153,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,"Coupletech Co., Ltd.",Китай,ems,1123.513546,224.70,0.75
154,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,"Accu-Tech Co., Ltd.",Китай,able,21140.895052,422.82,0.50


In [391]:
df_otgruz.tail()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель
308,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able,7283.079358,364.15,0.5
309,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd,4696.795535,469.68,0.5
310,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd,7749.647258,276.77,0.5
311,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd,2785.623414,146.61,0.5
312,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз,7416.069284,494.40,0.5


In [392]:
df_otgruz.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_otgruz2024.xlsx')  

In [393]:
df_dostavka_by_contract.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_dostavka_by_contract2024.xlsx')  

In [394]:
# 
df_contracts_2.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_contracts_2_2024.xlsx')  

In [136]:
# LOAD DATA
df_dostavka_by_contract = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_dostavka_by_contract2024.xlsx')
df_otgruz = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_otgruz2024.xlsx')
df_contracts_2 = pd.read_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_contracts_2_2024.xlsx')

In [137]:
# if data loaded
df_dostavka_by_contract = df_dostavka_by_contract[['№ инвойса (для РБ, РФ в скобках)','сумма, $']]
df_dostavka_by_contract.head()

,"№ инвойса (для РБ, РФ в скобках)","сумма, $"
0,(23-071),13.792670
1,(23-087),165.902782
2,(24-004),14.270570
3,23-008-4,972.067889
4,23-031-dc,63.853904


In [138]:
# if data loaded
df_otgruz = df_otgruz.drop(columns=['Unnamed: 0'])
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,"бух, $","бух, $/шт",Произв. расходы - множитель
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,439.189827,219.59,1.00
1,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225,ЗАО «ФОТЭК»,РБ,115.075662,57.54,1.00
2,22-01-2024,24-001-1,11-01-2024,1,новая продукция 08сч. - RFGM-40.68-350 rev.STD...,1685.0,05-04-2023,1,3653.61,0,"Coupletech Co., Ltd.",Китай,1260.126233,1260.13,1.00
3,22-01-2024,24-001-2,11-01-2024,1,РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600,"Coupletech Co., Ltd.",Китай,1123.513546,224.70,0.75
4,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650,"Accu-Tech Co., Ltd.",Китай,21140.895052,422.82,0.50


In [80]:
df_otgruz.tail(10)

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель
212,25-04-2024,24-037,25-04-2024,3,QBU-BT-10kV rev.STD 1.4 (№0705-0707),1875.0,25-04-2024,3,2036.20,1800.0,"LNG OPTICS, INC",Китай,able,628.049721,209.35,0.75
213,03-05-2024,24-038,26-04-2024,1,HVSW-04-4kV-WR rev.STD 1.0 (№0075),1880.0,02-05-2024,1,1919.48,900.0,"Coupletech Co., Ltd.",Китай,ems,589.701997,589.70,1.00
214,03-05-2024,24-038,26-04-2024,1,HVPS-300-4kV rev.STD 1.10 (№0047),1881.0,02-05-2024,1,691.93,350.0,"Coupletech Co., Ltd.",Китай,ems,212.574501,212.57,1.00
215,03-05-2024,24-038,26-04-2024,1,QBY-4001 новая продукция 08сч. - QBY-4001 (№0...,1879.0,30-04-2024,1,738.22,700.0,"Coupletech Co., Ltd.",Китай,ems,226.851454,226.85,1.00
216,15-05-2024,(24-034),05-04-2024,10,РСА-40-1000V-PD rev.STD 3.3 (№1001-1010) (45$ ...,1882.0,06-05-2024,10,16393.03,1295.0,ООО «Медицинские Оптические Технологии»,РФ,dpd,5080.432020,508.04,0.50
217,17-05-2024,24-029-1,25-04-2024,50,NBU-1012 rev.POT 1.2 NEL (№2402-2451),1872.0,17-04-2024,50,46756.21,600.0,GUANGZHOU POTENT MEDICAL EQUIPMENT \nJOINT-STO...,Китай,neo,14274.961837,285.50,0.50
218,20-05-2024,(24-039),25-04-2024,3,РСА-40-700V-PD rev.STD 1.7 (№1011-1013),1887.0,17-05-2024,3,4904.79,1800.0,ЗАО «СОЛАР ЛС»,РБ,самовывоз,1524.979013,508.33,0.75
219,29-05-2024,(24-032),19-03-2024,4,LDD-1500 100A-15V rev.STD 1.0 (№0170-0173) (В ...,1893.0,28-05-2024,4,5136.98,1600.0,ООО «ЛАЗЕРНЫЙ ЦЕНТР»,РФ,dpd,1619.425617,404.86,0.75
220,31-05-2024,24-040-1,06-05-2024,8,PDD-500 150A-45V rev.Venus 1.6 (№1500-1512),1894.0,29-05-2024,13,13540.07,600.0,HEALX - MEDICAL LTD,Израиль,ems,4275.766571,328.91,0.50
221,31-05-2024,24-040-2,06-05-2024,5,PDD-500 150A-45V rev.Venus 1.6 (№1500-1512),1894.0,29-05-2024,13,13540.07,600.0,HEALX - MEDICAL LTD,Израиль,ems,4275.766571,328.91,0.50


In [140]:
# if data loaded
df_contracts_2 = df_contracts_2.drop(columns=['Unnamed: 0'])
df_contracts_2

,"№ инвойса (для РБ, РФ в скобках)",Валюта,"Факт.сумма, $"
0,(23-071),rub,1158.287233
1,(23-087),rub,25896.961439
2,(23-090),byn,448.990354
3,(24-010),rub,1595.877612
4,(24-012),rub,2217.336321
5,(24-018),rub,23920.288273
6,(24-019),byn,3980.483587
7,23-008-4,cny,30292.603100
8,23-008-4,usd,5455.800000
9,23-066-5,cny,4561.986267


In [395]:
df_otgruz['Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)'] = df_otgruz[
    'Цена в инвойсе (дата инвойса-контракта), $/ шт'] * df_otgruz[
    'кол-во в инвойсе, шт.']

In [396]:
df_contracts_2 = df_contracts_2[['№ инвойса (для РБ, РФ в скобках)', 'Факт.сумма, $']
                               ].groupby('№ инвойса (для РБ, РФ в скобках)', as_index=False).sum()

In [397]:
df_contracts_2.tail(40)

,"№ инвойса (для РБ, РФ в скобках)","Факт.сумма, $"
45,24-016,4756.007794
46,24-020,2127.738262
47,24-023,2171.415230
48,24-025,32625.008006
49,24-026,1563.753024
50,24-027,28205.326343
51,24-029-1,30511.786968
52,24-029-2,30591.070417
53,24-029-3,30387.868857
54,24-029-4,31628.068084


In [398]:
df_otgruz.head(3)

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
150,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242,215.39,1.0,0.0
151,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662,57.54,1.0,450.0
152,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233,1260.13,1.0,0.0


In [399]:
df_otgruz.tail(10)

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
303,05-11-2024,24-085,01-11-2024,1,HVSW-04-2kV rev.STD 1.5 (0080),1961.0,18-10-2024,1,1938.30,2000.0,"Coupletech Co., Ltd.",Китай,able,585.253178,585.25,1.0,2000.0
304,05-11-2024,24-085,01-11-2024,1,HVPS-300-2kV rev.STD 1.4 (0055),1960.0,18-10-2024,1,724.90,500.0,"Coupletech Co., Ltd.",Китай,able,218.877382,218.88,1.0,500.0
305,05-11-2024,24-085,01-11-2024,2,QBY-4010 Разработка 08сч. - QBY-4010 rev.STD 1...,1953.0,09-10-2024,2,1561.38,1500.0,"Coupletech Co., Ltd.",Китай,able,475.277000,237.64,1.0,3000.0
306,05-11-2024,24-085,01-11-2024,1,RFGM-6.78-800 STD 1.2.1 (0012),1968.0,31-10-2024,1,2394.29,1050.0,"Coupletech Co., Ltd.",Китай,able,721.497665,721.50,1.0,1050.0
307,05-11-2024,24-008-2,03-09-2024,30,Медицинский зарядный модуль РСА-20-1200V-CD re...,1929.0,25-07-2024,49,54772.13,700.0,"Accu-Tech Co., Ltd.",Китай,able,17506.354077,357.27,0.5,21000.0
308,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able,7283.079358,364.15,0.5,14000.0
309,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd,4696.795535,469.68,0.5,7000.0
310,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd,7749.647258,276.77,0.5,5600.0
311,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd,2785.623414,146.61,0.5,3150.0
312,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз,7416.069284,494.40,0.5,11600.0


In [400]:
df_otgruz.columns

Index(['Дата отгрузки', '№ инвойса (для РБ, РФ в скобках)',
       'Дата контракта/инвойса/спецификации', 'кол-во в инвойсе, шт.',
       'Код в бух. + серийные номера', '№ выпуска', 'Дата выпуска',
       'кол-во в выпуске, шт.', 'общая цена выпуска, BYN',
       'Цена в инвойсе (дата инвойса-контракта), $/ шт', 'Контрагент',
       'Страна', 'Перевозчик', 'бух, $', 'бух, $/шт',
       'Произв. расходы - множитель',
       'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)'],
      dtype='object')

In [401]:
# отсечь несделанные отгрузки. или без выпуска. ЕСЛИ ЗАБЫЛИ ЭТО СДЕЛАТЬ ВНАЧАЛЕ!
# df_otgruz = df_otgruz.iloc[0:-3]

In [402]:
df_otgruz.tail()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
308,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able,7283.079358,364.15,0.5,14000.0
309,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd,4696.795535,469.68,0.5,7000.0
310,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd,7749.647258,276.77,0.5,5600.0
311,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd,2785.623414,146.61,0.5,3150.0
312,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз,7416.069284,494.40,0.5,11600.0


In [403]:
df_otgruz_by_contract = df_otgruz[
    ['№ инвойса (для РБ, РФ в скобках)', 
     'кол-во в инвойсе, шт.', 
     'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)',
    'Страна']
].groupby(['Страна','№ инвойса (для РБ, РФ в скобках)'], as_index=False).sum()

In [404]:
df_otgruz_by_contract.columns

Index(['Страна', '№ инвойса (для РБ, РФ в скобках)', 'кол-во в инвойсе, шт.',
       'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)'],
      dtype='object')

In [405]:
df_otgruz_by_contract.head(30)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
0,Израиль,24-007,7,4200.000
1,Израиль,24-030,7,4200.000
2,Израиль,24-040-1,8,4800.000
3,Израиль,24-040-2,5,3000.000
4,Израиль,24-047,32,22400.000
5,Индия,23-069-dc,1,0.000
6,Индия,24-033,22,16150.000
7,Индия,24-046,27,22125.000
8,Индия,24-062,2,0.000
9,Индия,24-072,4,4000.000


In [406]:
df_otgruz_by_contract.tail(10)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
78,РФ,(24-063),28,23450.0000
79,РФ,(24-064),1,1800.0000
80,РФ,(24-073),2,1000.0000
81,РФ,(24-075),1,1100.0000
82,РФ,(24-086),21,15750.0000
83,США,24-054,1,528.0000
84,США,24-060,1,759.9576
85,Тайвань,23-031-dc,2,0.0000
86,Тайвань,24-052,1,0.0000
87,Тайвань,24-083,4,4000.0000


In [129]:
#df_contracts_2[['№ инвойса (для РБ, РФ в скобках)', 'Факт.сумма, $']].tail(30)

In [407]:
# filtered_modul_df = report_dict_df.merge(self.df_from_file, on='Артикул', how='left')
df_otgruz_by_contract = df_otgruz_by_contract.merge(df_contracts_2[
    ['№ инвойса (для РБ, РФ в скобках)', 'Факт.сумма, $']], on='№ инвойса (для РБ, РФ в скобках)', how='left')

In [408]:
# на ремонтах и бесплатных образцах факт.сумма будет NAN - ее нужно заменить на 0
df_otgruz_by_contract.head(3)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $"
0,Израиль,24-007,7,4200.0,4676.683591
1,Израиль,24-030,7,4200.0,4453.142239
2,Израиль,24-040-1,8,4800.0,4972.496451


In [409]:
df_otgruz_by_contract[df_otgruz_by_contract['№ инвойса (для РБ, РФ в скобках)'] == '23-016-dc']

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $"
13,Китай,23-016-dc,2,0.0,0.0


In [410]:
# df[['a', 'b']] = df[['a','b']].fillna(value=0)
df_otgruz_by_contract[['Факт.сумма, $']] = df_otgruz_by_contract[['Факт.сумма, $']].fillna(value=0)

In [411]:
df_otgruz_by_contract[df_otgruz_by_contract['№ инвойса (для РБ, РФ в скобках)'] == '23-031-dc']

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $"
85,Тайвань,23-031-dc,2,0.0,0.0


In [412]:
df_otgruz_by_contract = df_otgruz_by_contract.merge(
    df_dostavka_by_contract, on='№ инвойса (для РБ, РФ в скобках)', how='left')

In [413]:
df_otgruz_by_contract.head(3)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $"
0,Израиль,24-007,7,4200.0,4676.683591,259.577805
1,Израиль,24-030,7,4200.0,4453.142239,147.884858
2,Израиль,24-040-1,8,4800.0,4972.496451,164.965167


In [414]:
df_otgruz_by_contract[['Цена всего инвойса без доставки, $']] = np.nan

In [415]:
df_otgruz_by_contract.loc[df_otgruz_by_contract['сумма, $'].notna(),
    'Цена всего инвойса без доставки, $'] = df_otgruz_by_contract[df_otgruz_by_contract['сумма, $'].notna()][
    'Факт.сумма, $'] - df_otgruz_by_contract[df_otgruz_by_contract['сумма, $'].notna()]['сумма, $'].fillna(0)

df_otgruz_by_contract[df_otgruz_by_contract['сумма, $'].notna()]

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $"
0,Израиль,24-007,7,4200.0000,4676.683591,259.577805,4417.105787
1,Израиль,24-030,7,4200.0000,4453.142239,147.884858,4305.257381
2,Израиль,24-040-1,8,4800.0000,4972.496451,164.965167,4807.531284
3,Израиль,24-040-2,5,3000.0000,3107.810282,115.731466,2992.078816
4,Израиль,24-047,32,22400.0000,23019.280266,677.263517,22342.016749
...,...,...,...,...,...,...,...
83,США,24-054,1,528.0000,728.285976,103.197422,625.088554
84,США,24-060,1,759.9576,728.285976,258.654601,469.631374
85,Тайвань,23-031-dc,2,0.0000,0.000000,63.853904,-63.853904
86,Тайвань,24-052,1,0.0000,0.000000,66.828315,-66.828315


In [416]:
# если доставки нет, то цену всего инвойса без доставки приравниваем полагаемой цене (сумма цены из инвойса по позиции)
df_otgruz_by_contract.loc[df_otgruz_by_contract['сумма, $'].isna(),
    'Цена всего инвойса без доставки, $'] = df_otgruz_by_contract[df_otgruz_by_contract['сумма, $'].isna()][
    'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)']

df_otgruz_by_contract[df_otgruz_by_contract['сумма, $'].isna()]

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $"
13,Китай,23-016-dc,2,0.0000,0.000000,NaN,0.0000
50,РБ,(22-188-3),1,1800.0000,1759.509315,NaN,1800.0000
51,РБ,(22-188-4),8,11600.0000,11600.000000,NaN,11600.0000
52,РБ,(23-090),2,450.0000,448.990354,NaN,450.0000
53,РБ,(24-014),2,9450.0000,9450.000929,NaN,9450.0000
54,РБ,(24-019),1,4000.0000,3980.483587,NaN,4000.0000
55,РБ,(24-039),3,5400.0000,5379.921332,NaN,5400.0000
56,РБ,(24-057),3,5400.0000,5427.034638,NaN,5400.0000
57,РБ,(24-058),2,1200.0000,1233.914218,NaN,1200.0000
58,РБ,(24-066),2,3500.0000,3482.019874,NaN,3500.0000


In [417]:
df_otgruz_by_contract.columns

Index(['Страна', '№ инвойса (для РБ, РФ в скобках)', 'кол-во в инвойсе, шт.',
       'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)',
       'Факт.сумма, $', 'сумма, $', 'Цена всего инвойса без доставки, $'],
      dtype='object')

In [418]:
# если РБ, то доставки нет в принципе. и цена без доставки = факт сумме
# df_otgruz_by_contract.loc[df_otgruz_by_contract['Валюта']=='byn',
#     'Цена всего инвойса без доставки, $'] = df_otgruz_by_contract[df_otgruz_by_contract['Валюта']=='byn'][
#     'Факт.сумма, $']

# df_otgruz_by_contract[df_otgruz_by_contract['Валюта']=='byn']

df_otgruz_by_contract.loc[df_otgruz_by_contract['Страна']=='РБ',
    'Цена всего инвойса без доставки, $'] = df_otgruz_by_contract[df_otgruz_by_contract['Страна']=='РБ'][
    'Факт.сумма, $']

df_otgruz_by_contract[df_otgruz_by_contract['Страна']=='РБ']

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $"
50,РБ,(22-188-3),1,1800.0000,1759.509315,NaN,1759.509315
51,РБ,(22-188-4),8,11600.0000,11600.000000,NaN,11600.000000
52,РБ,(23-090),2,450.0000,448.990354,NaN,448.990354
53,РБ,(24-014),2,9450.0000,9450.000929,NaN,9450.000929
54,РБ,(24-019),1,4000.0000,3980.483587,NaN,3980.483587
55,РБ,(24-039),3,5400.0000,5379.921332,NaN,5379.921332
56,РБ,(24-057),3,5400.0000,5427.034638,NaN,5427.034638
57,РБ,(24-058),2,1200.0000,1233.914218,NaN,1233.914218
58,РБ,(24-066),2,3500.0000,3482.019874,NaN,3482.019874
59,РБ,(24-069),1,1764.2643,1786.223278,NaN,1786.223278


In [419]:
df_otgruz_by_contract.head(3)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $"
0,Израиль,24-007,7,4200.0,4676.683591,259.577805,4417.105787
1,Израиль,24-030,7,4200.0,4453.142239,147.884858,4305.257381
2,Израиль,24-040-1,8,4800.0,4972.496451,164.965167,4807.531284


In [420]:
#df_otgruz_by_contract[df_otgruz_by_contract['№ инвойса (для РБ, РФ в скобках)'] == '(23-039)']

In [421]:
# если доставки нет, то факт цена за ед = полаг.цена за ед (а для РБ то, что оплатили)
df_otgruz_by_contract['Общ.бонус, $'] = df_otgruz_by_contract[
    'Цена всего инвойса без доставки, $'] - df_otgruz_by_contract[
    'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)']

In [422]:
df_otgruz_by_contract.head(3)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $","Общ.бонус, $"
0,Израиль,24-007,7,4200.0,4676.683591,259.577805,4417.105787,217.105787
1,Израиль,24-030,7,4200.0,4453.142239,147.884858,4305.257381,105.257381
2,Израиль,24-040-1,8,4800.0,4972.496451,164.965167,4807.531284,7.531284


In [423]:
df_otgruz_by_contract['Бонус на ед., $'] = df_otgruz_by_contract[
    'Общ.бонус, $'] / df_otgruz_by_contract[
    'кол-во в инвойсе, шт.']

In [424]:
df_otgruz_by_contract.head(3)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $","Общ.бонус, $","Бонус на ед., $"
0,Израиль,24-007,7,4200.0,4676.683591,259.577805,4417.105787,217.105787,31.015112
1,Израиль,24-030,7,4200.0,4453.142239,147.884858,4305.257381,105.257381,15.036769
2,Израиль,24-040-1,8,4800.0,4972.496451,164.965167,4807.531284,7.531284,0.941411


In [425]:
df_otgruz_by_contract.head(30)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $","Общ.бонус, $","Бонус на ед., $"
0,Израиль,24-007,7,4200.000,4676.683591,259.577805,4417.105787,217.105787,31.015112
1,Израиль,24-030,7,4200.000,4453.142239,147.884858,4305.257381,105.257381,15.036769
2,Израиль,24-040-1,8,4800.000,4972.496451,164.965167,4807.531284,7.531284,0.941411
3,Израиль,24-040-2,5,3000.000,3107.810282,115.731466,2992.078816,-7.921184,-1.584237
4,Израиль,24-047,32,22400.000,23019.280266,677.263517,22342.016749,-57.983251,-1.811977
5,Индия,23-069-dc,1,0.000,0.000000,276.966403,-276.966403,-276.966403,-276.966403
6,Индия,24-033,22,16150.000,16772.935271,414.383105,16358.552166,208.552166,9.479644
7,Индия,24-046,27,22125.000,22513.338460,578.417774,21934.920686,-190.079314,-7.039975
8,Индия,24-062,2,0.000,0.000000,269.914575,-269.914575,-269.914575,-134.957287
9,Индия,24-072,4,4000.000,4223.977035,267.337239,3956.639796,-43.360204,-10.840051


In [426]:
df_otgruz_by_contract.tail(10)

,Страна,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $","сумма, $","Цена всего инвойса без доставки, $","Общ.бонус, $","Бонус на ед., $"
78,РФ,(24-063),28,23450.0000,24285.923838,95.409541,24190.514298,740.514298,26.446939
79,РФ,(24-064),1,1800.0000,1880.550841,29.316994,1851.233846,51.233846,51.233846
80,РФ,(24-073),2,1000.0000,1061.800607,21.504380,1040.296227,40.296227,20.148113
81,РФ,(24-075),1,1100.0000,1184.437800,23.225915,1161.211885,61.211885,61.211885
82,РФ,(24-086),21,15750.0000,16727.162593,NaN,15750.000000,0.000000,0.000000
83,США,24-054,1,528.0000,728.285976,103.197422,625.088554,97.088554,97.088554
84,США,24-060,1,759.9576,728.285976,258.654601,469.631374,-290.326226,-290.326226
85,Тайвань,23-031-dc,2,0.0000,0.000000,63.853904,-63.853904,-63.853904,-31.926952
86,Тайвань,24-052,1,0.0000,0.000000,66.828315,-66.828315,-66.828315,-66.828315
87,Тайвань,24-083,4,4000.0000,4405.266020,333.349757,4071.916263,71.916263,17.979066


In [427]:
df_otgruz.head(3)

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
150,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242,215.39,1.0,0.0
151,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662,57.54,1.0,450.0
152,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233,1260.13,1.0,0.0


In [428]:
df_otgruz = df_otgruz.merge(df_otgruz_by_contract[[
    '№ инвойса (для РБ, РФ в скобках)', 
    'Бонус на ед., $']], on='№ инвойса (для РБ, РФ в скобках)', how='left')

In [429]:
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Бонус на ед., $"
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242,215.39,1.00,0.0,-31.926952
1,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662,57.54,1.00,450.0,-0.504823
2,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233,1260.13,1.00,0.0,0.000000
3,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,"Coupletech Co., Ltd.",Китай,ems,1123.513546,224.70,0.75,600.0,39.254687
4,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,"Accu-Tech Co., Ltd.",Китай,able,21140.895052,422.82,0.50,1300.0,-28.994014


In [430]:
df_otgruz.tail()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Бонус на ед., $"
158,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able,7283.079358,364.15,0.5,14000.0,-17.997839
159,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd,4696.795535,469.68,0.5,7000.0,0.000000
160,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd,7749.647258,276.77,0.5,5600.0,0.000000
161,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd,2785.623414,146.61,0.5,3150.0,0.000000
162,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз,7416.069284,494.40,0.5,11600.0,0.000000


In [431]:
df_otgruz['Факт.цена за шт./$'] = df_otgruz['Цена в инвойсе (дата инвойса-контракта), $/ шт'] + df_otgruz['Бонус на ед., $']

In [432]:
df_otgruz['Факт.цена за шт./$'] = df_otgruz['Факт.цена за шт./$'].apply(lambda x: round(Decimal(f'{x}'), 2))

In [433]:
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Бонус на ед., $",Факт.цена за шт./$
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242,215.39,1.00,0.0,-31.926952,-31.93
1,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662,57.54,1.00,450.0,-0.504823,224.50
2,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233,1260.13,1.00,0.0,0.000000,0.00
3,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,"Coupletech Co., Ltd.",Китай,ems,1123.513546,224.70,0.75,600.0,39.254687,639.25
4,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,"Accu-Tech Co., Ltd.",Китай,able,21140.895052,422.82,0.50,1300.0,-28.994014,621.01


In [434]:
df_otgruz.tail()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Бонус на ед., $",Факт.цена за шт./$
158,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able,7283.079358,364.15,0.5,14000.0,-17.997839,682.00
159,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd,4696.795535,469.68,0.5,7000.0,0.000000,1000.00
160,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd,7749.647258,276.77,0.5,5600.0,0.000000,800.00
161,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd,2785.623414,146.61,0.5,3150.0,0.000000,450.00
162,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз,7416.069284,494.40,0.5,11600.0,0.000000,1450.00


In [435]:
df_otgruz.dtypes

Дата отгрузки                                                    object
№ инвойса (для РБ, РФ в скобках)                                 object
Дата контракта/инвойса/спецификации                              object
кол-во в инвойсе, шт.                                             int64
Код в бух. + серийные номера                                     object
№ выпуска                                                       float64
Дата выпуска                                                     object
кол-во в выпуске, шт.                                             int64
общая цена выпуска, BYN                                         float64
Цена в инвойсе (дата инвойса-контракта), $/ шт                  float64
Контрагент                                                       object
Страна                                                           object
Перевозчик                                                       object
бух, $                                                          

In [436]:
df_otgruz['Период отгрузки'] = df_otgruz['Дата отгрузки'].apply(lambda x: '-'.join(str(x).split('-')[1:]))

In [437]:
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Бонус на ед., $",Факт.цена за шт./$,Период отгрузки
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,"FOUNDER TECHNOLOGY CO., LTD",Тайвань,ems,430.783242,215.39,1.00,0.0,-31.926952,-31.93,01-2024
1,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,ЗАО «ФОТЭК»,РБ,самовывоз,115.075662,57.54,1.00,450.0,-0.504823,224.50,01-2024
2,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,"Coupletech Co., Ltd.",Китай,ems,1260.126233,1260.13,1.00,0.0,0.000000,0.00,01-2024
3,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,"Coupletech Co., Ltd.",Китай,ems,1123.513546,224.70,0.75,600.0,39.254687,639.25,01-2024
4,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,"Accu-Tech Co., Ltd.",Китай,able,21140.895052,422.82,0.50,1300.0,-28.994014,621.01,01-2024


In [438]:
df_otgruz.tail()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",Контрагент,Страна,Перевозчик,"бух, $","бух, $/шт",Произв. расходы - множитель,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Бонус на ед., $",Факт.цена за шт./$,Период отгрузки
158,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,"Accu-Tech Co., Ltd.",Китай,able,7283.079358,364.15,0.5,14000.0,-17.997839,682.00,11-2024
159,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,"ООО ""МеЛСиТек""",РФ,dpd,4696.795535,469.68,0.5,7000.0,0.000000,1000.00,11-2024
160,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,"ООО ""МеЛСиТек""",РФ,dpd,7749.647258,276.77,0.5,5600.0,0.000000,800.00,11-2024
161,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,"ООО ""МеЛСиТек""",РФ,dpd,2785.623414,146.61,0.5,3150.0,0.000000,450.00,11-2024
162,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,ЗАО «СЛС Прайм Технолоджи»,РБ,самовывоз,7416.069284,494.40,0.5,11600.0,0.000000,1450.00,11-2024


In [439]:
# df_otgruz['бух, $/шт'] = df_otgruz['бух, $/шт'].apply(lambda x: round(Decimal(f'{x}'), 2))
# df_otgruz['бух, $/шт'] = df_otgruz['бух, $/шт'].apply(lambda x: round(Decimal(f'{x}'), 2))

In [440]:
# Произв. расходы на единицу
df_otgruz['Произв. расходы на единицу'] = df_otgruz['Произв. расходы - множитель'] * df_otgruz['бух, $/шт'].astype('float')

In [441]:
# Валовая прибыль на единицу
df_otgruz['Валовая прибыль на единицу'] = df_otgruz['Факт.цена за шт./$'].astype('float') - df_otgruz['бух, $/шт'].astype('float') - df_otgruz['Произв. расходы на единицу']

In [442]:
# Произв. расходы - сумма
df_otgruz['Произв. расходы - сумма'] = df_otgruz['кол-во в инвойсе, шт.'] * df_otgruz['Произв. расходы на единицу']

In [443]:
# Валовая прибыль - сумма
df_otgruz['Валовая прибыль - сумма'] = df_otgruz['кол-во в инвойсе, шт.'] * df_otgruz['Валовая прибыль на единицу']

In [444]:
# Компоненты
df_otgruz['Компоненты'] = df_otgruz['кол-во в инвойсе, шт.'] * df_otgruz['бух, $/шт']

In [445]:
# Продажи  (Факт. сумма, $)
df_otgruz['Продажи (факт)'] = df_otgruz['кол-во в инвойсе, шт.'] * df_otgruz['Факт.цена за шт./$']

In [446]:
# gross margine
df_otgruz['gross margine'] = df_otgruz['Валовая прибыль - сумма'] / df_otgruz['Продажи (факт)'].astype('float')

In [447]:
df_otgruz['Продажи (факт)'] = df_otgruz['Продажи (факт)'].astype('float')

In [448]:
df_otgruz['Факт.цена за шт./$'] = df_otgruz['Факт.цена за шт./$'].astype('float')

In [449]:
df_otgruz.head()

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,...,-31.926952,-31.93,01-2024,215.390,-462.710,430.780,-925.420,430.78,-63.86,14.491387
1,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,...,-0.504823,224.50,01-2024,57.540,109.420,115.080,218.840,115.08,449.00,0.487394
2,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,...,0.000000,0.00,01-2024,1260.130,-2520.260,1260.130,-2520.260,1260.13,0.00,-inf
3,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,...,39.254687,639.25,01-2024,168.525,246.025,168.525,246.025,224.70,639.25,0.384865
4,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,...,-28.994014,621.01,01-2024,211.410,-13.220,422.820,-26.440,845.64,1242.02,-0.021288


In [450]:
df_otgruz.tail(10)

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
153,05-11-2024,24-085,01-11-2024,1,HVSW-04-2kV rev.STD 1.5 (0080),1961.0,18-10-2024,1,1938.30,2000.0,...,4.227062,2004.23,11-2024,585.250,833.730,585.250,833.730,585.25,2004.23,0.415985
154,05-11-2024,24-085,01-11-2024,1,HVPS-300-2kV rev.STD 1.4 (0055),1960.0,18-10-2024,1,724.90,500.0,...,4.227062,504.23,11-2024,218.880,66.470,218.880,66.470,218.88,504.23,0.131825
155,05-11-2024,24-085,01-11-2024,2,QBY-4010 Разработка 08сч. - QBY-4010 rev.STD 1...,1953.0,09-10-2024,2,1561.38,1500.0,...,4.227062,1504.23,11-2024,237.640,1028.950,475.280,2057.900,475.28,3008.46,0.684038
156,05-11-2024,24-085,01-11-2024,1,RFGM-6.78-800 STD 1.2.1 (0012),1968.0,31-10-2024,1,2394.29,1050.0,...,4.227062,1054.23,11-2024,721.500,-388.770,721.500,-388.770,721.50,1054.23,-0.368772
157,05-11-2024,24-008-2,03-09-2024,30,Медицинский зарядный модуль РСА-20-1200V-CD re...,1929.0,25-07-2024,49,54772.13,700.0,...,-17.997839,682.00,11-2024,178.635,146.095,5359.050,4382.850,10718.10,20460.00,0.214216
158,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,...,-17.997839,682.00,11-2024,182.075,135.775,3641.500,2715.500,7283.00,13640.00,0.199084
159,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,...,0.000000,1000.00,11-2024,234.840,295.480,1643.880,2068.360,3287.76,7000.00,0.295480
160,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,...,0.000000,800.00,11-2024,138.385,384.845,968.695,2693.915,1937.39,5600.00,0.481056
161,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,...,0.000000,450.00,11-2024,73.305,230.085,513.135,1610.595,1026.27,3150.00,0.511300
162,12-11-2024,(22-188-4),23-10-2024,8,РСА-40-1500V-PD (1048-1062),1971.0,11-11-2024,15,24704.41,1450.0,...,0.000000,1450.00,11-2024,247.200,708.400,1977.600,5667.200,3955.20,11600.00,0.488552


In [451]:
df_otgruz.columns

Index(['Дата отгрузки', '№ инвойса (для РБ, РФ в скобках)',
       'Дата контракта/инвойса/спецификации', 'кол-во в инвойсе, шт.',
       'Код в бух. + серийные номера', '№ выпуска', 'Дата выпуска',
       'кол-во в выпуске, шт.', 'общая цена выпуска, BYN',
       'Цена в инвойсе (дата инвойса-контракта), $/ шт', 'Контрагент',
       'Страна', 'Перевозчик', 'бух, $', 'бух, $/шт',
       'Произв. расходы - множитель',
       'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)',
       'Бонус на ед., $', 'Факт.цена за шт./$', 'Период отгрузки',
       'Произв. расходы на единицу', 'Валовая прибыль на единицу',
       'Произв. расходы - сумма', 'Валовая прибыль - сумма', 'Компоненты',
       'Продажи (факт)', 'gross margine'],
      dtype='object')

In [452]:
df_otgruz[df_otgruz['Компоненты'].isna()]

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine


In [453]:
df_otgruz[df_otgruz['Продажи (факт)'] < 1]

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,...,-31.926952,-31.93,01-2024,215.39,-462.71,430.78,-925.42,430.78,-63.86,14.491387
2,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,...,0.000000,0.00,01-2024,1260.13,-2520.26,1260.13,-2520.26,1260.13,0.00,-inf
10,23-01-2024,(23-071-rem),28-08-2023,1,"QBU-mini-SP-4016 ремонт по гарантии для ЛЛС,РФ",NaN,22-01-2024,1,7.46,0.0,...,-13.792670,-13.79,01-2024,2.36,-18.51,2.36,-18.51,2.36,-13.79,1.342277
81,13-06-2024,23-069-dc,13-06-2024,1,LDD-1500-15A-150V-HF новая продукция 08сч. - L...,1898.0,13-06-2024,1,1230.61,0.0,...,-276.966403,-276.97,06-2024,385.92,-1048.81,385.92,-1048.81,385.92,-276.97,3.786728
87,01-07-2024,24-052,28-06-2024,1,RFGM-6.78-800 ремонт HVSW-04-4kV rev.STD 1.7 (...,1714.0,17-05-2023,1,1549.92,0.0,...,-66.828315,-66.83,07-2024,531.19,-1129.21,531.19,-1129.21,531.19,-66.83,16.896753
115,13-08-2024,24-062,26-07-2024,2,LDD-1500 новая продукция 08сч. - LDD-1500-15A-...,1935.0,12-08-2024,2,1010.92,0.0,...,-134.957287,-134.96,08-2024,161.76,-458.48,323.52,-916.96,323.52,-269.92,3.397155
151,05-11-2024,23-016-dc,05-11-2024,2,NaN,NaN,05-11-2024,2,0.00,0.0,...,0.000000,0.00,11-2024,0.00,0.00,0.00,0.00,0.00,0.00,NaN


In [454]:
df_otgruz[df_otgruz['Компоненты'] < 1]

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
151,05-11-2024,23-016-dc,05-11-2024,2,NaN,NaN,05-11-2024,2,0.0,0.0,...,0.0,0.0,11-2024,0.0,0.0,0.0,0.0,0.00,0.0,NaN


In [138]:
# отсечь несделанные отгрузки. или без выпуска. ЕСЛИ ЗАБЫЛИ ЭТО СДЕЛАТЬ ВНАЧАЛЕ!
# df_otgruz = df_otgruz.iloc[0:-3]

In [455]:
df_otgruz[df_otgruz['№ инвойса (для РБ, РФ в скобках)'] == '23-031-dc']

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,...,-31.926952,-31.93,01-2024,215.39,-462.71,430.78,-925.42,430.78,-63.86,14.491387


In [456]:
df_otgruz[df_otgruz['№ инвойса (для РБ, РФ в скобках)'] == '24-033']

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
55,23-04-2024,24-033,29-03-2024,20,NBU-1012 rev.ALL 2.2.IWW NEL (№2382-2401),1871.0,11-04-2024,20,21692.32,807.5,...,9.479644,816.98,04-2024,166.28,318.14,3325.60,6362.8,6651.20,16339.60,0.389410
56,23-04-2024,24-033,29-03-2024,2,LDD-1500-15A-150V-HV новая продукция 08сч. - L...,1873.0,30-04-2024,2,2199.44,0.0,...,9.479644,9.48,04-2024,337.94,-666.40,675.88,-1332.8,675.88,18.96,-70.295359


In [457]:
#--------------------------------
# ПЕРЕСЧЕТ СУММЫ КОТРАКТА-ИНВОЙСА В $

# прочесть файл
contr_df = pd.read_excel(
    'D:/!HRYTS_FOLDER/2023-REPORT/2023_contracts.xlsx', 
    sheet_name='Лист2', 
    usecols='A,B,C'
)
contr_df.head(10)

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма
0,24-090,eur,2307.00
1,24-089,usd,40600.00
2,(22-188-5),byn,33823.86
3,(24-088),byn,13962.76
4,(24-087),rub,292404.00
5,(24-086),rub,1632162.00
6,24-085,cny,153225.00
7,23-016-dc,usd,0.00
8,24-084,eur,9276.00
9,24-083,cny,31208.50


In [458]:
contr_dates_df = df_otgruz[[
    'Период отгрузки', 
    '№ инвойса (для РБ, РФ в скобках)', 
    'Дата контракта/инвойса/спецификации'
]].drop_duplicates()
contr_dates_df.shape

(88, 3)

In [459]:
contr_df_1 = contr_df.merge(
    contr_dates_df, on='№ инвойса (для РБ, РФ в скобках)', how='left'
)

contr_df_1

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Период отгрузки,Дата контракта/инвойса/спецификации
0,24-090,eur,2307.00,NaN,NaN
1,24-089,usd,40600.00,NaN,NaN
2,(22-188-5),byn,33823.86,NaN,NaN
3,(24-088),byn,13962.76,NaN,NaN
4,(24-087),rub,292404.00,NaN,NaN
...,...,...,...,...,...
106,23-066-5,usd,26265.00,02-2024,22-01-2024
107,23-066-5,usd,4635.00,02-2024,22-01-2024
108,23-093-1,usd,30900.00,03-2024,19-02-2024
109,23-093-2,usd,30900.00,04-2024,06-03-2024


In [460]:
contr_df_1 = contr_df_1.dropna()
contr_df_1

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Период отгрузки,Дата контракта/инвойса/спецификации
5,(24-086),rub,1632162.0,11-2024,04-11-2024
6,24-085,cny,153225.0,11-2024,01-11-2024
7,23-016-dc,usd,0.0,11-2024,05-11-2024
8,24-084,eur,9276.0,10-2024,28-10-2024
9,24-083,cny,31208.5,10-2024,25-10-2024
...,...,...,...,...,...
106,23-066-5,usd,26265.0,02-2024,22-01-2024
107,23-066-5,usd,4635.0,02-2024,22-01-2024
108,23-093-1,usd,30900.0,03-2024,19-02-2024
109,23-093-2,usd,30900.0,04-2024,06-03-2024


In [461]:
contr_df_1['Сумма, $'] = contr_df_1.apply(
    lambda x: to_usd_by_date(x['Дата контракта/инвойса/спецификации'], x['Сумма'], x['Валюта']), axis=1)
contr_df_1

,"№ инвойса (для РБ, РФ в скобках)",Валюта,Сумма,Период отгрузки,Дата контракта/инвойса/спецификации,"Сумма, $"
5,(24-086),rub,1632162.0,11-2024,04-11-2024,16830.887184
6,24-085,cny,153225.0,11-2024,01-11-2024,21587.897141
7,23-016-dc,usd,0.0,11-2024,05-11-2024,0.000000
8,24-084,eur,9276.0,10-2024,28-10-2024,10039.409313
9,24-083,cny,31208.5,10-2024,25-10-2024,4405.266020
...,...,...,...,...,...,...
106,23-066-5,usd,26265.0,02-2024,22-01-2024,26265.000000
107,23-066-5,usd,4635.0,02-2024,22-01-2024,4635.000000
108,23-093-1,usd,30900.0,03-2024,19-02-2024,30900.000000
109,23-093-2,usd,30900.0,04-2024,06-03-2024,30900.000000


In [462]:
contr_df_2 = contr_df_1[
    ['№ инвойса (для РБ, РФ в скобках)', 
     'Период отгрузки', 
     'Дата контракта/инвойса/спецификации',
    'Сумма, $']
].groupby(['№ инвойса (для РБ, РФ в скобках)', 
     'Период отгрузки', 
     'Дата контракта/инвойса/спецификации'], as_index=False).sum()
contr_df_2

,"№ инвойса (для РБ, РФ в скобках)",Период отгрузки,Дата контракта/инвойса/спецификации,"Сумма, $"
0,(22-188-3),10-2024,30-09-2024,1800.000000
1,(22-188-4),11-2024,23-10-2024,11600.000000
2,(23-071-rem),01-2024,28-08-2023,0.000000
3,(23-087),01-2024,16-11-2023,25354.048139
4,(23-090),01-2024,29-11-2023,450.003223
...,...,...,...,...
80,24-076,10-2024,07-10-2024,4115.595991
81,24-079,10-2024,15-10-2024,1430.059346
82,24-083,10-2024,25-10-2024,4405.266020
83,24-084,10-2024,28-10-2024,10039.409313


In [463]:
df_otgruz_compo = df_otgruz[[
    '№ инвойса (для РБ, РФ в скобках)', 
    'Компоненты',
'Дата отгрузки']].groupby(['№ инвойса (для РБ, РФ в скобках)','Дата отгрузки'], as_index=False, dropna=False).sum()
df_otgruz_compo

,"№ инвойса (для РБ, РФ в скобках)",Дата отгрузки,Компоненты
0,(22-188-3),21-10-2024,535.24
1,(22-188-4),12-11-2024,3955.20
2,(23-071-rem),23-01-2024,2.36
3,(23-087),29-01-2024,9158.10
4,(23-090),15-01-2024,115.08
...,...,...,...
83,24-076,07-10-2024,702.36
84,24-079,16-10-2024,624.91
85,24-083,30-10-2024,1922.20
86,24-084,30-10-2024,2950.90


In [464]:
# df_otgruz = df_otgruz.merge(df_otgruz_by_contract[[
#     '№ инвойса (для РБ, РФ в скобках)', 
#     'Бонус на ед., $']], on='№ инвойса (для РБ, РФ в скобках)', how='left')
contr_df_3 = contr_df_2.merge(df_otgruz_compo[[
    '№ инвойса (для РБ, РФ в скобках)', 
    'Компоненты']], on='№ инвойса (для РБ, РФ в скобках)', how='left')
contr_df_3

,"№ инвойса (для РБ, РФ в скобках)",Период отгрузки,Дата контракта/инвойса/спецификации,"Сумма, $",Компоненты
0,(22-188-3),10-2024,30-09-2024,1800.000000,535.24
1,(22-188-4),11-2024,23-10-2024,11600.000000,3955.20
2,(23-071-rem),01-2024,28-08-2023,0.000000,2.36
3,(23-087),01-2024,16-11-2023,25354.048139,9158.10
4,(23-090),01-2024,29-11-2023,450.003223,115.08
...,...,...,...,...,...
80,24-076,10-2024,07-10-2024,4115.595991,702.36
81,24-079,10-2024,15-10-2024,1430.059346,624.91
82,24-083,10-2024,25-10-2024,4405.266020,1922.20
83,24-084,10-2024,28-10-2024,10039.409313,2950.90


In [465]:
df_otgruz.columns

Index(['Дата отгрузки', '№ инвойса (для РБ, РФ в скобках)',
       'Дата контракта/инвойса/спецификации', 'кол-во в инвойсе, шт.',
       'Код в бух. + серийные номера', '№ выпуска', 'Дата выпуска',
       'кол-во в выпуске, шт.', 'общая цена выпуска, BYN',
       'Цена в инвойсе (дата инвойса-контракта), $/ шт', 'Контрагент',
       'Страна', 'Перевозчик', 'бух, $', 'бух, $/шт',
       'Произв. расходы - множитель',
       'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)',
       'Бонус на ед., $', 'Факт.цена за шт./$', 'Период отгрузки',
       'Произв. расходы на единицу', 'Валовая прибыль на единицу',
       'Произв. расходы - сумма', 'Валовая прибыль - сумма', 'Компоненты',
       'Продажи (факт)', 'gross margine'],
      dtype='object')

In [466]:
df_otgruz_fact = df_otgruz[[
    '№ инвойса (для РБ, РФ в скобках)', 
    'Продажи (факт)',
    'Дата отгрузки',
    'Контрагент',
]].groupby(['№ инвойса (для РБ, РФ в скобках)','Дата отгрузки', 'Контрагент'], as_index=False, dropna=False).sum()
df_otgruz_fact.head()

,"№ инвойса (для РБ, РФ в скобках)",Дата отгрузки,Контрагент,Продажи (факт)
0,(22-188-3),21-10-2024,АДВИН Смарт Фэктори ЗАО,1759.51
1,(22-188-4),12-11-2024,ЗАО «СЛС Прайм Технолоджи»,11600.00
2,(23-071-rem),23-01-2024,"АО ""ЛЛС""",-13.79
3,(23-087),29-01-2024,"АО ""ПЛЛТ""",25731.00
4,(23-090),15-01-2024,ЗАО «ФОТЭК»,449.00


In [467]:
df_otgruz[df_otgruz['№ инвойса (для РБ, РФ в скобках)']=='(23-071-rem)']

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
10,23-01-2024,(23-071-rem),28-08-2023,1,"QBU-mini-SP-4016 ремонт по гарантии для ЛЛС,РФ",NaN,22-01-2024,1,7.46,0.0,...,-13.79267,-13.79,01-2024,2.36,-18.51,2.36,-18.51,2.36,-13.79,1.342277


In [468]:
contr_df_4 = contr_df_3.merge(df_otgruz_fact[[
    '№ инвойса (для РБ, РФ в скобках)', 
    'Продажи (факт)',
    'Контрагент'
]], on='№ инвойса (для РБ, РФ в скобках)', how='left')
contr_df_4

,"№ инвойса (для РБ, РФ в скобках)",Период отгрузки,Дата контракта/инвойса/спецификации,"Сумма, $",Компоненты,Продажи (факт),Контрагент
0,(22-188-3),10-2024,30-09-2024,1800.000000,535.24,1759.51,АДВИН Смарт Фэктори ЗАО
1,(22-188-4),11-2024,23-10-2024,11600.000000,3955.20,11600.00,ЗАО «СЛС Прайм Технолоджи»
2,(23-071-rem),01-2024,28-08-2023,0.000000,2.36,-13.79,"АО ""ЛЛС"""
3,(23-087),01-2024,16-11-2023,25354.048139,9158.10,25731.00,"АО ""ПЛЛТ"""
4,(23-090),01-2024,29-11-2023,450.003223,115.08,449.00,ЗАО «ФОТЭК»
...,...,...,...,...,...,...,...
80,24-076,10-2024,07-10-2024,4115.595991,702.36,3837.27,Coreray Corp.
81,24-079,10-2024,15-10-2024,1430.059346,624.91,969.81,"Coupletech Co., Ltd."
82,24-083,10-2024,25-10-2024,4405.266020,1922.20,4071.92,"FOUNDER TECHNOLOGY CO., LTD"
83,24-084,10-2024,28-10-2024,10039.409313,2950.90,9777.20,Allengers Global Healthcare Pvt Ltd


In [469]:
df_otgruz_inv = df_otgruz[[
    '№ инвойса (для РБ, РФ в скобках)', 
    'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)',
]].groupby(['№ инвойса (для РБ, РФ в скобках)'], as_index=False, dropna=False).sum()
df_otgruz_inv.head()

,"№ инвойса (для РБ, РФ в скобках)","Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
0,(22-188-3),1800.0
1,(22-188-4),11600.0
2,(23-071-rem),0.0
3,(23-087),24000.0
4,(23-090),450.0


In [470]:
contr_df_5 = contr_df_4.merge(df_otgruz_inv, on='№ инвойса (для РБ, РФ в скобках)', how='left')
contr_df_5

,"№ инвойса (для РБ, РФ в скобках)",Период отгрузки,Дата контракта/инвойса/спецификации,"Сумма, $",Компоненты,Продажи (факт),Контрагент,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)"
0,(22-188-3),10-2024,30-09-2024,1800.000000,535.24,1759.51,АДВИН Смарт Фэктори ЗАО,1800.0000
1,(22-188-4),11-2024,23-10-2024,11600.000000,3955.20,11600.00,ЗАО «СЛС Прайм Технолоджи»,11600.0000
2,(23-071-rem),01-2024,28-08-2023,0.000000,2.36,-13.79,"АО ""ЛЛС""",0.0000
3,(23-087),01-2024,16-11-2023,25354.048139,9158.10,25731.00,"АО ""ПЛЛТ""",24000.0000
4,(23-090),01-2024,29-11-2023,450.003223,115.08,449.00,ЗАО «ФОТЭК»,450.0000
...,...,...,...,...,...,...,...,...
80,24-076,10-2024,07-10-2024,4115.595991,702.36,3837.27,Coreray Corp.,4115.5959
81,24-079,10-2024,15-10-2024,1430.059346,624.91,969.81,"Coupletech Co., Ltd.",960.0000
82,24-083,10-2024,25-10-2024,4405.266020,1922.20,4071.92,"FOUNDER TECHNOLOGY CO., LTD",4000.0000
83,24-084,10-2024,28-10-2024,10039.409313,2950.90,9777.20,Allengers Global Healthcare Pvt Ltd,9740.6950


In [471]:
df_contracts_2.head()

,"№ инвойса (для РБ, РФ в скобках)","Факт.сумма, $"
0,(22-188-3),1759.509315
1,(22-188-4),11600.000000
2,(23-087),25896.961439
3,(23-090),448.990354
4,(24-004),650.782320


In [472]:
contr_df_6 = contr_df_5.merge(df_contracts_2, on='№ инвойса (для РБ, РФ в скобках)', how='left')
contr_df_6

,"№ инвойса (для РБ, РФ в скобках)",Период отгрузки,Дата контракта/инвойса/спецификации,"Сумма, $",Компоненты,Продажи (факт),Контрагент,"Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)","Факт.сумма, $"
0,(22-188-3),10-2024,30-09-2024,1800.000000,535.24,1759.51,АДВИН Смарт Фэктори ЗАО,1800.0000,1759.509315
1,(22-188-4),11-2024,23-10-2024,11600.000000,3955.20,11600.00,ЗАО «СЛС Прайм Технолоджи»,11600.0000,11600.000000
2,(23-071-rem),01-2024,28-08-2023,0.000000,2.36,-13.79,"АО ""ЛЛС""",0.0000,NaN
3,(23-087),01-2024,16-11-2023,25354.048139,9158.10,25731.00,"АО ""ПЛЛТ""",24000.0000,25896.961439
4,(23-090),01-2024,29-11-2023,450.003223,115.08,449.00,ЗАО «ФОТЭК»,450.0000,448.990354
...,...,...,...,...,...,...,...,...,...
80,24-076,10-2024,07-10-2024,4115.595991,702.36,3837.27,Coreray Corp.,4115.5959,4115.595991
81,24-079,10-2024,15-10-2024,1430.059346,624.91,969.81,"Coupletech Co., Ltd.",960.0000,1430.059346
82,24-083,10-2024,25-10-2024,4405.266020,1922.20,4071.92,"FOUNDER TECHNOLOGY CO., LTD",4000.0000,4405.266020
83,24-084,10-2024,28-10-2024,10039.409313,2950.90,9777.20,Allengers Global Healthcare Pvt Ltd,9740.6950,10039.409313


In [473]:
contr_df_6.rename({
    'Сумма, $': 'Сумма контракта, $', 
    'Компоненты': 'Себестоимость, $',
    'Продажи (факт)': 'Деньги, полученные по контракту, после вычета накладных расходов, $',
    'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)': 'Деньги, которые мы собирались получить, после вычета накладных расходов, $',
    'Факт.сумма, $': 'Сумма по контракту, которую заплатили, $',
}, axis=1, inplace=True)
contr_df_6.columns

Index(['№ инвойса (для РБ, РФ в скобках)', 'Период отгрузки',
       'Дата контракта/инвойса/спецификации', 'Сумма контракта, $',
       'Себестоимость, $',
       'Деньги, полученные по контракту, после вычета накладных расходов, $',
       'Контрагент',
       'Деньги, которые мы собирались получить, после вычета накладных расходов, $',
       'Сумма по контракту, которую заплатили, $'],
      dtype='object')

In [474]:
contr_df_6 = contr_df_6.fillna(0)

In [475]:
contr_df_6.head()

,"№ инвойса (для РБ, РФ в скобках)",Период отгрузки,Дата контракта/инвойса/спецификации,"Сумма контракта, $","Себестоимость, $","Деньги, полученные по контракту, после вычета накладных расходов, $",Контрагент,"Деньги, которые мы собирались получить, после вычета накладных расходов, $","Сумма по контракту, которую заплатили, $"
0,(22-188-3),10-2024,30-09-2024,1800.000000,535.24,1759.51,АДВИН Смарт Фэктори ЗАО,1800.0,1759.509315
1,(22-188-4),11-2024,23-10-2024,11600.000000,3955.20,11600.00,ЗАО «СЛС Прайм Технолоджи»,11600.0,11600.000000
2,(23-071-rem),01-2024,28-08-2023,0.000000,2.36,-13.79,"АО ""ЛЛС""",0.0,0.000000
3,(23-087),01-2024,16-11-2023,25354.048139,9158.10,25731.00,"АО ""ПЛЛТ""",24000.0,25896.961439
4,(23-090),01-2024,29-11-2023,450.003223,115.08,449.00,ЗАО «ФОТЭК»,450.0,448.990354


In [476]:
contr_df_7 = contr_df_6[['№ инвойса (для РБ, РФ в скобках)', 'Контрагент', 'Период отгрузки',
                        'Себестоимость, $',
                        'Сумма контракта, $',
                        'Сумма по контракту, которую заплатили, $',
                        'Деньги, которые мы собирались получить, после вычета накладных расходов, $',
                        'Деньги, полученные по контракту, после вычета накладных расходов, $'
                        ]]
contr_df_7.head()

,"№ инвойса (для РБ, РФ в скобках)",Контрагент,Период отгрузки,"Себестоимость, $","Сумма контракта, $","Сумма по контракту, которую заплатили, $","Деньги, которые мы собирались получить, после вычета накладных расходов, $","Деньги, полученные по контракту, после вычета накладных расходов, $"
0,(22-188-3),АДВИН Смарт Фэктори ЗАО,10-2024,535.24,1800.000000,1759.509315,1800.0,1759.51
1,(22-188-4),ЗАО «СЛС Прайм Технолоджи»,11-2024,3955.20,11600.000000,11600.000000,11600.0,11600.00
2,(23-071-rem),"АО ""ЛЛС""",01-2024,2.36,0.000000,0.000000,0.0,-13.79
3,(23-087),"АО ""ПЛЛТ""",01-2024,9158.10,25354.048139,25896.961439,24000.0,25731.00
4,(23-090),ЗАО «ФОТЭК»,01-2024,115.08,450.003223,448.990354,450.0,449.00


In [477]:
contr_df_7

,"№ инвойса (для РБ, РФ в скобках)",Контрагент,Период отгрузки,"Себестоимость, $","Сумма контракта, $","Сумма по контракту, которую заплатили, $","Деньги, которые мы собирались получить, после вычета накладных расходов, $","Деньги, полученные по контракту, после вычета накладных расходов, $"
0,(22-188-3),АДВИН Смарт Фэктори ЗАО,10-2024,535.24,1800.000000,1759.509315,1800.0000,1759.51
1,(22-188-4),ЗАО «СЛС Прайм Технолоджи»,11-2024,3955.20,11600.000000,11600.000000,11600.0000,11600.00
2,(23-071-rem),"АО ""ЛЛС""",01-2024,2.36,0.000000,0.000000,0.0000,-13.79
3,(23-087),"АО ""ПЛЛТ""",01-2024,9158.10,25354.048139,25896.961439,24000.0000,25731.00
4,(23-090),ЗАО «ФОТЭК»,01-2024,115.08,450.003223,448.990354,450.0000,449.00
...,...,...,...,...,...,...,...,...
80,24-076,Coreray Corp.,10-2024,702.36,4115.595991,4115.595991,4115.5959,3837.27
81,24-079,"Coupletech Co., Ltd.",10-2024,624.91,1430.059346,1430.059346,960.0000,969.81
82,24-083,"FOUNDER TECHNOLOGY CO., LTD",10-2024,1922.20,4405.266020,4405.266020,4000.0000,4071.92
83,24-084,Allengers Global Healthcare Pvt Ltd,10-2024,2950.90,10039.409313,10039.409313,9740.6950,9777.20


In [478]:
contr_df_7['Себестоимость, $'] = contr_df_7['Себестоимость, $'].astype('float')

In [479]:
contr_df_7.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_otgruz_invoice_2024_oct.xlsx',float_format="%.2f", index=False)

In [480]:
df_otgruz

,Дата отгрузки,"№ инвойса (для РБ, РФ в скобках)",Дата контракта/инвойса/спецификации,"кол-во в инвойсе, шт.",Код в бух. + серийные номера,№ выпуска,Дата выпуска,"кол-во в выпуске, шт.","общая цена выпуска, BYN","Цена в инвойсе (дата инвойса-контракта), $/ шт",...,"Бонус на ед., $",Факт.цена за шт./$,Период отгрузки,Произв. расходы на единицу,Валовая прибыль на единицу,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,Продажи (факт),gross margine
0,11-01-2024,23-031-dc,14-12-2023,2,RF-MNF rev.STD 1.1 (№0003-0004),1798.0,04-12-2024,2,1395.35,0.0,...,-31.926952,-31.93,01-2024,215.390,-462.710,430.780,-925.420,430.78,-63.86,14.491387
1,15-01-2024,(23-090),29-11-2023,2,SBZ-2008 rev.STD 1.3 (№2920-2921),1817.0,21-12-2023,2,365.02,225.0,...,-0.504823,224.50,01-2024,57.540,109.420,115.080,218.840,115.08,449.00,0.487394
2,22-01-2024,24-001-1,11-01-2024,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,1685.0,05-04-2023,1,3653.61,0.0,...,0.000000,0.00,01-2024,1260.130,-2520.260,1260.130,-2520.260,1260.13,0.00,-inf
3,22-01-2024,24-001-2,11-01-2024,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),1576.0,18-10-2022,5,2832.49,600.0,...,39.254687,639.25,01-2024,168.525,246.025,168.525,246.025,224.70,639.25,0.384865
4,23-01-2024,23-008-4,08-01-2024,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),1733.0,06-07-2023,50,64340.20,650.0,...,-28.994014,621.01,01-2024,211.410,-13.220,422.820,-26.440,845.64,1242.02,-0.021288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,05-11-2024,24-008-2,03-09-2024,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),1944.0,25-09-2024,20,23466.81,700.0,...,-17.997839,682.00,11-2024,182.075,135.775,3641.500,2715.500,7283.00,13640.00,0.199084
159,06-11-2024,(24-086),04-11-2024,7,PCA-25-800V-PD (2947-2956),1940.0,13-09-2024,10,14979.49,1000.0,...,0.000000,1000.00,11-2024,234.840,295.480,1643.880,2068.360,3287.76,7000.00,0.295480
160,06-11-2024,(24-086),04-11-2024,7,NBU-1012 STD 1.7 (2654-2681),1939.0,13-09-2024,28,24715.95,800.0,...,0.000000,800.00,11-2024,138.385,384.845,968.695,2693.915,1937.39,5600.00,0.481056
161,06-11-2024,(24-086),04-11-2024,7,LSCB-100ms (0458-0476),1942.0,17-09-2024,19,8889.76,450.0,...,0.000000,450.00,11-2024,73.305,230.085,513.135,1610.595,1026.27,3150.00,0.511300


In [481]:
df_otgruz[['Период отгрузки', '№ инвойса (для РБ, РФ в скобках)',  
     'Продажи (факт)']][df_otgruz['Период отгрузки'] == '09-2024']

,Период отгрузки,"№ инвойса (для РБ, РФ в скобках)",Продажи (факт)
118,09-2024,24-008-1,20589.90
119,09-2024,24-008-1,686.33
120,09-2024,24-008-1,13040.27
121,09-2024,24-029-4,31126.50
122,09-2024,24-056,12410.80
123,09-2024,(24-014),9450.00
124,09-2024,24-070,14424.30
125,09-2024,24-072,3956.64
126,09-2024,(24-069),1786.22
127,09-2024,(24-049),5577.48


In [482]:
df_otgruz[['Период отгрузки',   
     'Продажи (факт)']][df_otgruz['Период отгрузки'] == '09-2024'].groupby(
    'Период отгрузки', as_index=False, dropna=False).sum()

,Период отгрузки,Продажи (факт)
0,09-2024,147822.71


In [483]:
# df_otgruz_report = df_otgruz[
#     ['Период отгрузки', 
#      'Продажи',
#      'Компоненты', 
#      'Произв. расходы - сумма', 
#      'Валовая прибыль - сумма']].groupby('Период отгрузки', as_index=False, dropna=True).sum()
df_otgruz_report_1 = df_otgruz[
    ['Период отгрузки',   
     'Продажи (факт)',
     #'Компоненты', 
     #'Произв. расходы - сумма', 
     #'Валовая прибыль - сумма',
    ]].groupby('Период отгрузки', as_index=False, dropna=False).sum()

In [484]:
# куда пропадают "КОМПОНЕНТЫ"  ???
df_otgruz_report_1.columns

Index(['Период отгрузки', 'Продажи (факт)'], dtype='object')

In [485]:
df_otgruz_report_1

,Период отгрузки,Продажи (факт)
0,01-2024,65835.40
1,02-2024,90916.23
2,03-2024,67045.37
3,04-2024,169155.79
4,05-2024,64993.94
5,06-2024,81285.54
6,07-2024,98607.56
7,08-2024,10198.40
8,09-2024,147822.71
9,10-2024,98698.24


In [486]:
df_otgruz_report_2 = df_otgruz[
    ['Период отгрузки',   
#      'Продажи (факт)',
     'Компоненты', 
     #'Произв. расходы - сумма', 
     #'Валовая прибыль - сумма',
    ]].groupby('Период отгрузки', as_index=False, dropna=False).sum()

In [487]:
df_otgruz_report_2.columns

Index(['Период отгрузки', 'Компоненты'], dtype='object')

In [488]:
df_otgruz_report_2

,Период отгрузки,Компоненты
0,01-2024,34761.55
1,02-2024,42805.33
2,03-2024,30881.21
3,04-2024,83219.94
4,05-2024,27819.21
5,06-2024,35943.95
6,07-2024,41494.38
7,08-2024,5105.06
8,09-2024,59144.90
9,10-2024,36763.23


In [489]:
df_otgruz_report_3 = df_otgruz[
    ['Период отгрузки',   
#      'Продажи (факт)',
#      'Компоненты', 
     'Произв. расходы - сумма', 
     'Валовая прибыль - сумма',
    ]].groupby('Период отгрузки', as_index=False, dropna=False).sum()

In [490]:
df_otgruz_report_3.columns

Index(['Период отгрузки', 'Произв. расходы - сумма',
       'Валовая прибыль - сумма'],
      dtype='object')

In [491]:
df_otgruz_report_3

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма
0,01-2024,18487.0250,12586.8250
1,02-2024,25024.4700,23086.4300
2,03-2024,16981.1950,19182.9650
3,04-2024,43835.9325,42099.9175
4,05-2024,15213.9125,21960.8175
5,06-2024,20849.2175,24492.3725
6,07-2024,24503.3825,32609.7975
7,08-2024,4704.4325,388.9075
8,09-2024,31457.7450,57220.0650
9,10-2024,20443.3950,41491.6150


In [492]:
df_otgruz_report_4 = df_otgruz_report_1.merge(df_otgruz_report_2, how='left', on='Период отгрузки')

In [493]:
df_otgruz_report_4

,Период отгрузки,Продажи (факт),Компоненты
0,01-2024,65835.40,34761.55
1,02-2024,90916.23,42805.33
2,03-2024,67045.37,30881.21
3,04-2024,169155.79,83219.94
4,05-2024,64993.94,27819.21
5,06-2024,81285.54,35943.95
6,07-2024,98607.56,41494.38
7,08-2024,10198.40,5105.06
8,09-2024,147822.71,59144.90
9,10-2024,98698.24,36763.23


In [494]:
df_otgruz_report = df_otgruz_report_3.merge(df_otgruz_report_4, how='left', on='Период отгрузки')

In [495]:
df_otgruz_report

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма,Продажи (факт),Компоненты
0,01-2024,18487.0250,12586.8250,65835.40,34761.55
1,02-2024,25024.4700,23086.4300,90916.23,42805.33
2,03-2024,16981.1950,19182.9650,67045.37,30881.21
3,04-2024,43835.9325,42099.9175,169155.79,83219.94
4,05-2024,15213.9125,21960.8175,64993.94,27819.21
5,06-2024,20849.2175,24492.3725,81285.54,35943.95
6,07-2024,24503.3825,32609.7975,98607.56,41494.38
7,08-2024,4704.4325,388.9075,10198.40,5105.06
8,09-2024,31457.7450,57220.0650,147822.71,59144.90
9,10-2024,20443.3950,41491.6150,98698.24,36763.23


In [496]:
df_otgruz_report.columns

Index(['Период отгрузки', 'Произв. расходы - сумма', 'Валовая прибыль - сумма',
       'Продажи (факт)', 'Компоненты'],
      dtype='object')

In [497]:
df_otgruz_report['gross margine'] = df_otgruz_report['Валовая прибыль - сумма'] / df_otgruz_report['Продажи (факт)']

In [498]:
df_otgruz_report

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма,Продажи (факт),Компоненты,gross margine
0,01-2024,18487.0250,12586.8250,65835.40,34761.55,0.191186
1,02-2024,25024.4700,23086.4300,90916.23,42805.33,0.253931
2,03-2024,16981.1950,19182.9650,67045.37,30881.21,0.286119
3,04-2024,43835.9325,42099.9175,169155.79,83219.94,0.248883
4,05-2024,15213.9125,21960.8175,64993.94,27819.21,0.337890
5,06-2024,20849.2175,24492.3725,81285.54,35943.95,0.301313
6,07-2024,24503.3825,32609.7975,98607.56,41494.38,0.330703
7,08-2024,4704.4325,388.9075,10198.40,5105.06,0.038134
8,09-2024,31457.7450,57220.0650,147822.71,59144.90,0.387086
9,10-2024,20443.3950,41491.6150,98698.24,36763.23,0.420389


In [499]:
df_otgruz_report.columns

Index(['Период отгрузки', 'Произв. расходы - сумма', 'Валовая прибыль - сумма',
       'Продажи (факт)', 'Компоненты', 'gross margine'],
      dtype='object')

In [500]:
df_otgruz_report.rename({
    'Продажи (факт)': 'Деньги, фактически полученные по контрактам (инвойсам), $', 
}, axis=1, inplace=True)
df_otgruz_report.columns

Index(['Период отгрузки', 'Произв. расходы - сумма', 'Валовая прибыль - сумма',
       'Деньги, фактически полученные по контрактам (инвойсам), $',
       'Компоненты', 'gross margine'],
      dtype='object')

In [501]:
df_otgruz_report

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма,"Деньги, фактически полученные по контрактам (инвойсам), $",Компоненты,gross margine
0,01-2024,18487.0250,12586.8250,65835.40,34761.55,0.191186
1,02-2024,25024.4700,23086.4300,90916.23,42805.33,0.253931
2,03-2024,16981.1950,19182.9650,67045.37,30881.21,0.286119
3,04-2024,43835.9325,42099.9175,169155.79,83219.94,0.248883
4,05-2024,15213.9125,21960.8175,64993.94,27819.21,0.337890
5,06-2024,20849.2175,24492.3725,81285.54,35943.95,0.301313
6,07-2024,24503.3825,32609.7975,98607.56,41494.38,0.330703
7,08-2024,4704.4325,388.9075,10198.40,5105.06,0.038134
8,09-2024,31457.7450,57220.0650,147822.71,59144.90,0.387086
9,10-2024,20443.3950,41491.6150,98698.24,36763.23,0.420389


In [502]:
contr_df_7.head()

,"№ инвойса (для РБ, РФ в скобках)",Контрагент,Период отгрузки,"Себестоимость, $","Сумма контракта, $","Сумма по контракту, которую заплатили, $","Деньги, которые мы собирались получить, после вычета накладных расходов, $","Деньги, полученные по контракту, после вычета накладных расходов, $"
0,(22-188-3),АДВИН Смарт Фэктори ЗАО,10-2024,535.24,1800.000000,1759.509315,1800.0,1759.51
1,(22-188-4),ЗАО «СЛС Прайм Технолоджи»,11-2024,3955.20,11600.000000,11600.000000,11600.0,11600.00
2,(23-071-rem),"АО ""ЛЛС""",01-2024,2.36,0.000000,0.000000,0.0,-13.79
3,(23-087),"АО ""ПЛЛТ""",01-2024,9158.10,25354.048139,25896.961439,24000.0,25731.00
4,(23-090),ЗАО «ФОТЭК»,01-2024,115.08,450.003223,448.990354,450.0,449.00


In [503]:
contr_df_8 = contr_df_7[
    ['Период отгрузки', 
     'Себестоимость, $',
     'Сумма контракта, $',
     'Сумма по контракту, которую заплатили, $', 
     'Деньги, которые мы собирались получить, после вычета накладных расходов, $', 
     'Деньги, полученные по контракту, после вычета накладных расходов, $',
    ]].groupby('Период отгрузки', as_index=False, dropna=False).sum()

In [504]:
contr_df_8.columns

Index(['Период отгрузки', 'Себестоимость, $', 'Сумма контракта, $',
       'Сумма по контракту, которую заплатили, $',
       'Деньги, которые мы собирались получить, после вычета накладных расходов, $',
       'Деньги, полученные по контракту, после вычета накладных расходов, $'],
      dtype='object')

In [505]:
contr_df_8

,Период отгрузки,"Себестоимость, $","Сумма контракта, $","Сумма по контракту, которую заплатили, $","Деньги, которые мы собирались получить, после вычета накладных расходов, $","Деньги, полученные по контракту, после вычета накладных расходов, $"
0,01-2024,34761.55,67379.855829,67243.564773,65800.0000,65835.40
1,02-2024,42805.33,93264.074123,92471.313036,89598.0000,90916.23
2,03-2024,30881.21,69573.891814,68201.219874,66500.0000,67045.37
3,04-2024,83219.94,174610.207164,172575.338073,167950.0000,169155.79
4,05-2024,27819.21,66843.360088,66103.185511,64500.0000,64993.94
5,06-2024,35558.03,83320.129551,83256.890759,80100.0000,81562.51
6,07-2024,41379.94,98304.481412,98585.880247,95013.0000,96330.54
7,08-2024,5105.06,11495.698294,11560.572197,10659.9576,10198.40
8,09-2024,53587.06,135703.959961,135889.371688,131864.2643,133512.79
9,10-2024,36763.23,101897.383332,100945.540994,98216.2909,98698.24


In [506]:
df_otgruz_report

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма,"Деньги, фактически полученные по контрактам (инвойсам), $",Компоненты,gross margine
0,01-2024,18487.0250,12586.8250,65835.40,34761.55,0.191186
1,02-2024,25024.4700,23086.4300,90916.23,42805.33,0.253931
2,03-2024,16981.1950,19182.9650,67045.37,30881.21,0.286119
3,04-2024,43835.9325,42099.9175,169155.79,83219.94,0.248883
4,05-2024,15213.9125,21960.8175,64993.94,27819.21,0.337890
5,06-2024,20849.2175,24492.3725,81285.54,35943.95,0.301313
6,07-2024,24503.3825,32609.7975,98607.56,41494.38,0.330703
7,08-2024,4704.4325,388.9075,10198.40,5105.06,0.038134
8,09-2024,31457.7450,57220.0650,147822.71,59144.90,0.387086
9,10-2024,20443.3950,41491.6150,98698.24,36763.23,0.420389


In [507]:
df_otgruz_report_fin = df_otgruz_report.merge(contr_df_8, how='left', on='Период отгрузки')

In [508]:
df_otgruz_report_fin

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма,"Деньги, фактически полученные по контрактам (инвойсам), $",Компоненты,gross margine,"Себестоимость, $","Сумма контракта, $","Сумма по контракту, которую заплатили, $","Деньги, которые мы собирались получить, после вычета накладных расходов, $","Деньги, полученные по контракту, после вычета накладных расходов, $"
0,01-2024,18487.0250,12586.8250,65835.40,34761.55,0.191186,34761.55,67379.855829,67243.564773,65800.0000,65835.40
1,02-2024,25024.4700,23086.4300,90916.23,42805.33,0.253931,42805.33,93264.074123,92471.313036,89598.0000,90916.23
2,03-2024,16981.1950,19182.9650,67045.37,30881.21,0.286119,30881.21,69573.891814,68201.219874,66500.0000,67045.37
3,04-2024,43835.9325,42099.9175,169155.79,83219.94,0.248883,83219.94,174610.207164,172575.338073,167950.0000,169155.79
4,05-2024,15213.9125,21960.8175,64993.94,27819.21,0.337890,27819.21,66843.360088,66103.185511,64500.0000,64993.94
5,06-2024,20849.2175,24492.3725,81285.54,35943.95,0.301313,35558.03,83320.129551,83256.890759,80100.0000,81562.51
6,07-2024,24503.3825,32609.7975,98607.56,41494.38,0.330703,41379.94,98304.481412,98585.880247,95013.0000,96330.54
7,08-2024,4704.4325,388.9075,10198.40,5105.06,0.038134,5105.06,11495.698294,11560.572197,10659.9576,10198.40
8,09-2024,31457.7450,57220.0650,147822.71,59144.90,0.387086,53587.06,135703.959961,135889.371688,131864.2643,133512.79
9,10-2024,20443.3950,41491.6150,98698.24,36763.23,0.420389,36763.23,101897.383332,100945.540994,98216.2909,98698.24


In [509]:
df_otgruz_report_fin.columns

Index(['Период отгрузки', 'Произв. расходы - сумма', 'Валовая прибыль - сумма',
       'Деньги, фактически полученные по контрактам (инвойсам), $',
       'Компоненты', 'gross margine', 'Себестоимость, $', 'Сумма контракта, $',
       'Сумма по контракту, которую заплатили, $',
       'Деньги, которые мы собирались получить, после вычета накладных расходов, $',
       'Деньги, полученные по контракту, после вычета накладных расходов, $'],
      dtype='object')

In [510]:
df_otgruz_report_fin = df_otgruz_report_fin[[
    'Период отгрузки',
    'Произв. расходы - сумма', 
    'Валовая прибыль - сумма',
    'Компоненты',
    'gross margine',
    'Сумма контракта, $',
    'Сумма по контракту, которую заплатили, $',
    'Деньги, которые мы собирались получить, после вычета накладных расходов, $',
    'Деньги, полученные по контракту, после вычета накладных расходов, $',
    'Деньги, фактически полученные по контрактам (инвойсам), $',      
]]

In [511]:
df_otgruz_report_fin.dtypes

Период отгрузки                                                                object
Произв. расходы - сумма                                                       float64
Валовая прибыль - сумма                                                       float64
Компоненты                                                                     object
gross margine                                                                 float64
Сумма контракта, $                                                            float64
Сумма по контракту, которую заплатили, $                                      float64
Деньги, которые мы собирались получить, после вычета накладных расходов, $    float64
Деньги, полученные по контракту, после вычета накладных расходов, $           float64
Деньги, фактически полученные по контрактам (инвойсам), $                     float64
dtype: object

In [512]:
df_otgruz_report_fin['Компоненты'] = df_otgruz_report_fin['Компоненты'].astype('float')

In [513]:
df_otgruz_report_fin['gross margine kviloryia'] = 100*(
    df_otgruz_report_fin['Сумма контракта, $'] - df_otgruz_report_fin['Компоненты']
)/df_otgruz_report_fin['Сумма контракта, $']

In [514]:
df_otgruz_report_fin.dtypes

Период отгрузки                                                                object
Произв. расходы - сумма                                                       float64
Валовая прибыль - сумма                                                       float64
Компоненты                                                                    float64
gross margine                                                                 float64
Сумма контракта, $                                                            float64
Сумма по контракту, которую заплатили, $                                      float64
Деньги, которые мы собирались получить, после вычета накладных расходов, $    float64
Деньги, полученные по контракту, после вычета накладных расходов, $           float64
Деньги, фактически полученные по контрактам (инвойсам), $                     float64
gross margine kviloryia                                                       float64
dtype: object

In [515]:
df_otgruz_report_fin

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,gross margine,"Сумма контракта, $","Сумма по контракту, которую заплатили, $","Деньги, которые мы собирались получить, после вычета накладных расходов, $","Деньги, полученные по контракту, после вычета накладных расходов, $","Деньги, фактически полученные по контрактам (инвойсам), $",gross margine kviloryia
0,01-2024,18487.0250,12586.8250,34761.55,0.191186,67379.855829,67243.564773,65800.0000,65835.40,65835.40,48.409581
1,02-2024,25024.4700,23086.4300,42805.33,0.253931,93264.074123,92471.313036,89598.0000,90916.23,90916.23,54.103088
2,03-2024,16981.1950,19182.9650,30881.21,0.286119,69573.891814,68201.219874,66500.0000,67045.37,67045.37,55.613795
3,04-2024,43835.9325,42099.9175,83219.94,0.248883,174610.207164,172575.338073,167950.0000,169155.79,169155.79,52.339590
4,05-2024,15213.9125,21960.8175,27819.21,0.337890,66843.360088,66103.185511,64500.0000,64993.94,64993.94,58.381491
5,06-2024,20849.2175,24492.3725,35943.95,0.301313,83320.129551,83256.890759,80100.0000,81562.51,81285.54,56.860425
6,07-2024,24503.3825,32609.7975,41494.38,0.330703,98304.481412,98585.880247,95013.0000,96330.54,98607.56,57.789941
7,08-2024,4704.4325,388.9075,5105.06,0.038134,11495.698294,11560.572197,10659.9576,10198.40,10198.40,55.591562
8,09-2024,31457.7450,57220.0650,59144.90,0.387086,135703.959961,135889.371688,131864.2643,133512.79,147822.71,56.416231
9,10-2024,20443.3950,41491.6150,36763.23,0.420389,101897.383332,100945.540994,98216.2909,98698.24,98698.24,63.921321


In [516]:
df_otgruz_report_fin.rename({
    'Сумма контракта, $': 'Сумма по контрактам (инвойсам), $', 
    'Сумма по контракту, которую заплатили, $': 'Сколько поступило нам на счет, $',
}, axis=1, inplace=True)

In [517]:
df_otgruz_report_fin

,Период отгрузки,Произв. расходы - сумма,Валовая прибыль - сумма,Компоненты,gross margine,"Сумма по контрактам (инвойсам), $","Сколько поступило нам на счет, $","Деньги, которые мы собирались получить, после вычета накладных расходов, $","Деньги, полученные по контракту, после вычета накладных расходов, $","Деньги, фактически полученные по контрактам (инвойсам), $",gross margine kviloryia
0,01-2024,18487.0250,12586.8250,34761.55,0.191186,67379.855829,67243.564773,65800.0000,65835.40,65835.40,48.409581
1,02-2024,25024.4700,23086.4300,42805.33,0.253931,93264.074123,92471.313036,89598.0000,90916.23,90916.23,54.103088
2,03-2024,16981.1950,19182.9650,30881.21,0.286119,69573.891814,68201.219874,66500.0000,67045.37,67045.37,55.613795
3,04-2024,43835.9325,42099.9175,83219.94,0.248883,174610.207164,172575.338073,167950.0000,169155.79,169155.79,52.339590
4,05-2024,15213.9125,21960.8175,27819.21,0.337890,66843.360088,66103.185511,64500.0000,64993.94,64993.94,58.381491
5,06-2024,20849.2175,24492.3725,35943.95,0.301313,83320.129551,83256.890759,80100.0000,81562.51,81285.54,56.860425
6,07-2024,24503.3825,32609.7975,41494.38,0.330703,98304.481412,98585.880247,95013.0000,96330.54,98607.56,57.789941
7,08-2024,4704.4325,388.9075,5105.06,0.038134,11495.698294,11560.572197,10659.9576,10198.40,10198.40,55.591562
8,09-2024,31457.7450,57220.0650,59144.90,0.387086,135703.959961,135889.371688,131864.2643,133512.79,147822.71,56.416231
9,10-2024,20443.3950,41491.6150,36763.23,0.420389,101897.383332,100945.540994,98216.2909,98698.24,98698.24,63.921321


In [518]:
df_otgruz_report_fin.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_otgruz_report-10-2024.xlsx',float_format="%.2f", index=False)  

In [519]:
df_otgruz.dtypes

Дата отгрузки                                                    object
№ инвойса (для РБ, РФ в скобках)                                 object
Дата контракта/инвойса/спецификации                              object
кол-во в инвойсе, шт.                                             int64
Код в бух. + серийные номера                                     object
№ выпуска                                                       float64
Дата выпуска                                                     object
кол-во в выпуске, шт.                                             int64
общая цена выпуска, BYN                                         float64
Цена в инвойсе (дата инвойса-контракта), $/ шт                  float64
Контрагент                                                       object
Страна                                                           object
Перевозчик                                                       object
бух, $                                                          

In [520]:
df_otgruz.columns

Index(['Дата отгрузки', '№ инвойса (для РБ, РФ в скобках)',
       'Дата контракта/инвойса/спецификации', 'кол-во в инвойсе, шт.',
       'Код в бух. + серийные номера', '№ выпуска', 'Дата выпуска',
       'кол-во в выпуске, шт.', 'общая цена выпуска, BYN',
       'Цена в инвойсе (дата инвойса-контракта), $/ шт', 'Контрагент',
       'Страна', 'Перевозчик', 'бух, $', 'бух, $/шт',
       'Произв. расходы - множитель',
       'Цена в инвойсе (дата инвойса-контракта), $ (по всей позиции)',
       'Бонус на ед., $', 'Факт.цена за шт./$', 'Период отгрузки',
       'Произв. расходы на единицу', 'Валовая прибыль на единицу',
       'Произв. расходы - сумма', 'Валовая прибыль - сумма', 'Компоненты',
       'Продажи (факт)', 'gross margine'],
      dtype='object')

In [521]:
df_otgruz_short = df_otgruz[[
                            'Дата отгрузки',
                            'Период отгрузки',
                            '№ инвойса (для РБ, РФ в скобках)',
                            'кол-во в инвойсе, шт.',
                            'Код в бух. + серийные номера',
                            'Контрагент',
                            'Страна', 
                            'Цена в инвойсе (дата инвойса-контракта), $/ шт',
                            'Факт.цена за шт./$',
                            'бух, $/шт',
                             'Продажи (факт)'
                            ]]

In [522]:
df_otgruz_short.rename({
    'Продажи (факт)': 'Деньги, полученные за эту строку в инвойсе (цена за шт * кол-во шт), $', 
}, axis=1, inplace=True)

In [523]:
df_otgruz_short

,Дата отгрузки,Период отгрузки,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.",Код в бух. + серийные номера,Контрагент,Страна,"Цена в инвойсе (дата инвойса-контракта), $/ шт",Факт.цена за шт./$,"бух, $/шт","Деньги, полученные за эту строку в инвойсе (цена за шт * кол-во шт), $"
0,11-01-2024,01-2024,23-031-dc,2,RF-MNF rev.STD 1.1 (№0003-0004),"FOUNDER TECHNOLOGY CO., LTD",Тайвань,0.0,-31.93,215.39,-63.86
1,15-01-2024,01-2024,(23-090),2,SBZ-2008 rev.STD 1.3 (№2920-2921),ЗАО «ФОТЭК»,РБ,225.0,224.50,57.54,449.00
2,22-01-2024,01-2024,24-001-1,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,"Coupletech Co., Ltd.",Китай,0.0,0.00,1260.13,0.00
3,22-01-2024,01-2024,24-001-2,1,RFGM РСА-10-300V-PD rev.STD 1.8.SPP (№8134-8138),"Coupletech Co., Ltd.",Китай,600.0,639.25,224.70,639.25
4,23-01-2024,01-2024,23-008-4,2,РСА-20-1200V-CD rev.LNG 3.6 (№2340-2389),"Accu-Tech Co., Ltd.",Китай,650.0,621.01,422.82,1242.02
...,...,...,...,...,...,...,...,...,...,...,...
158,05-11-2024,11-2024,24-008-2,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),"Accu-Tech Co., Ltd.",Китай,700.0,682.00,364.15,13640.00
159,06-11-2024,11-2024,(24-086),7,PCA-25-800V-PD (2947-2956),"ООО ""МеЛСиТек""",РФ,1000.0,1000.00,469.68,7000.00
160,06-11-2024,11-2024,(24-086),7,NBU-1012 STD 1.7 (2654-2681),"ООО ""МеЛСиТек""",РФ,800.0,800.00,276.77,5600.00
161,06-11-2024,11-2024,(24-086),7,LSCB-100ms (0458-0476),"ООО ""МеЛСиТек""",РФ,450.0,450.00,146.61,3150.00


In [524]:
# df_contracts_1[(df_contracts_1['Дата прихода на счет'].isnull())&(df_contracts_1['Сумма'].notnull())]
df_otgruz_short[(df_otgruz_short['№ инвойса (для РБ, РФ в скобках)']=='(24-018)')]

,Дата отгрузки,Период отгрузки,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.",Код в бух. + серийные номера,Контрагент,Страна,"Цена в инвойсе (дата инвойса-контракта), $/ шт",Факт.цена за шт./$,"бух, $/шт","Деньги, полученные за эту строку в инвойсе (цена за шт * кол-во шт), $"
32,29-02-2024,02-2024,(24-018),5,PDD-300 100A-30V rev.TMS 1.2 (№0801-0805),"АО ""ПЛЛТ""",РФ,700.0,723.58,322.12,3617.90
33,29-02-2024,02-2024,(24-018),8,РСА-10-450V-PD rev.TMS 1.6 (№8592-8606),"АО ""ПЛЛТ""",РФ,550.0,573.58,234.76,4588.64
34,29-02-2024,02-2024,(24-018),8,NBU-1012 rev.TMS 1.9 Solar (№2297-2311),"АО ""ПЛЛТ""",РФ,800.0,823.58,290.72,6588.64
35,29-02-2024,02-2024,(24-018),16,CB-450V-14mF rev.STD 1.2 (№1044-1073),"АО ""ПЛЛТ""",РФ,225.0,248.58,67.15,3977.28
36,29-02-2024,02-2024,(24-018),8,PDD-500 100A-33V rev.TMS 1.4 (№1468-1482),"АО ""ПЛЛТ""",РФ,600.0,623.58,282.81,4988.64


In [525]:
df_otgruz_short[(df_otgruz_short['№ инвойса (для РБ, РФ в скобках)']=='23-031-dc')]

,Дата отгрузки,Период отгрузки,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.",Код в бух. + серийные номера,Контрагент,Страна,"Цена в инвойсе (дата инвойса-контракта), $/ шт",Факт.цена за шт./$,"бух, $/шт","Деньги, полученные за эту строку в инвойсе (цена за шт * кол-во шт), $"
0,11-01-2024,01-2024,23-031-dc,2,RF-MNF rev.STD 1.1 (№0003-0004),"FOUNDER TECHNOLOGY CO., LTD",Тайвань,0.0,-31.93,215.39,-63.86


In [526]:
df_otgruz_short[(df_otgruz_short['№ инвойса (для РБ, РФ в скобках)']=='24-001-1')]

,Дата отгрузки,Период отгрузки,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.",Код в бух. + серийные номера,Контрагент,Страна,"Цена в инвойсе (дата инвойса-контракта), $/ шт",Факт.цена за шт./$,"бух, $/шт","Деньги, полученные за эту строку в инвойсе (цена за шт * кол-во шт), $"
2,22-01-2024,01-2024,24-001-1,1,RFGM-40.68-350 новая продукция 08сч. - RFGM-40...,"Coupletech Co., Ltd.",Китай,0.0,0.0,1260.13,0.0


In [527]:
df_otgruz_short['бух, $/шт'] = df_otgruz['бух, $/шт'].astype('float')

In [528]:
df_otgruz_short['Компоненты'] = df_otgruz['Компоненты'].astype('float')

In [529]:
df_otgruz_short.dtypes

Дата отгрузки                                                              object
Период отгрузки                                                            object
№ инвойса (для РБ, РФ в скобках)                                           object
кол-во в инвойсе, шт.                                                       int64
Код в бух. + серийные номера                                               object
Контрагент                                                                 object
Страна                                                                     object
Цена в инвойсе (дата инвойса-контракта), $/ шт                            float64
Факт.цена за шт./$                                                        float64
бух, $/шт                                                                 float64
Деньги, полученные за эту строку в инвойсе (цена за шт * кол-во шт), $    float64
Компоненты                                                                float64
dtype: object

In [530]:
df_otgruz_short['70%'] = 100*df_otgruz_short['бух, $/шт']/df_otgruz_short['Цена в инвойсе (дата инвойса-контракта), $/ шт']

In [531]:
df_otgruz_short.tail()

,Дата отгрузки,Период отгрузки,"№ инвойса (для РБ, РФ в скобках)","кол-во в инвойсе, шт.",Код в бух. + серийные номера,Контрагент,Страна,"Цена в инвойсе (дата инвойса-контракта), $/ шт",Факт.цена за шт./$,"бух, $/шт","Деньги, полученные за эту строку в инвойсе (цена за шт * кол-во шт), $",Компоненты,70%
158,05-11-2024,11-2024,24-008-2,20,РСА-20-1200V-CD rev.LNG 3.10 (2957-2976),"Accu-Tech Co., Ltd.",Китай,700.0,682.0,364.15,13640.0,7283.00,52.021429
159,06-11-2024,11-2024,(24-086),7,PCA-25-800V-PD (2947-2956),"ООО ""МеЛСиТек""",РФ,1000.0,1000.0,469.68,7000.0,3287.76,46.968000
160,06-11-2024,11-2024,(24-086),7,NBU-1012 STD 1.7 (2654-2681),"ООО ""МеЛСиТек""",РФ,800.0,800.0,276.77,5600.0,1937.39,34.596250
161,06-11-2024,11-2024,(24-086),7,LSCB-100ms (0458-0476),"ООО ""МеЛСиТек""",РФ,450.0,450.0,146.61,3150.0,1026.27,32.580000
162,12-11-2024,11-2024,(22-188-4),8,РСА-40-1500V-PD (1048-1062),ЗАО «СЛС Прайм Технолоджи»,РБ,1450.0,1450.0,494.40,11600.0,3955.20,34.096552


In [532]:
df_otgruz_short.to_excel('D:/!HRYTS_FOLDER/2023-REPORT/df_otgruz_short-oct2024+.xlsx',float_format="%.2f", index=False)  